# Comparison of Similarity Metrics

This appendix compares the effect of all similarity metric implementations of the textdistance library [[TeDi](./A_References.ipynb#tedi)] for some example strings pair combinations for each feature to be calculated. The comparison is based on the goldstandard data and is the basis for deciding the similarity metrics for each feature to be used.

## Table of Contents

- [Data Takeover](#Data-Takeover)
- [Functions for Similarity Metrics Analysis](#Functions-for-Similarity-Metrics-Analysis)
- [Similarity Metric Assessments](#Similarity-Metric-Assessments)
    - [coordinate](#coordinate)
    - [corporate](#corporate)
    - [doi](#doi)
    - [edition](#edition)
    - [exactDate](#exactDate)
    - [format](#format)
    - [isbn](#isbn)
    - [ismn](#ismn)
    - [musicid](#musicid)
    - [part](#part)
    - [person](#person)
    - [pubinit](#pubinit)
    - [scale](#scale)
    - [ttlfull](#ttlfull)
    - [volumes](#volumes)

## Data Takeover

As a first step, the training data set as a result of chapter [Goldstandard and Data Preparation](./2_GoldstandardDataPreparation.ipynb) is read. Some sample strings of this data set will be used for the comparison and assessment of the different metrics implementations.

In [1]:
import os
import pandas as pd
import pickle as pk
import bz2
import _pickle as cPickle

path_goldstandard = './daten_goldstandard'

# Restore metadata so far
with open(os.path.join(path_goldstandard, 'columns_metadata.pkl'), 'rb') as handle:
    columns_metadata_dict = pk.load(handle)

# Restore DataFrame with features from compressed pickle file
with bz2.BZ2File((os.path.join(
    path_goldstandard, 'feature_base_df.pkl')), 'rb') as file:
    df_feature_base = cPickle.load(file)

# Extend display to number of columns of DataFrame
pd.options.display.max_columns = len(df_feature_base.columns)

df_feature_base.head()

,035liste_x,035liste_y,century_x,century_y,coordinate_E_x,coordinate_E_y,coordinate_N_x,coordinate_N_y,coordinate_x,coordinate_y,corporate_110_x,corporate_110_y,corporate_710_x,corporate_710_y,corporate_full_x,corporate_full_y,decade_x,decade_y,docid_x,docid_y,doi_x,doi_y,duplicates,edition_x,edition_y,exactDate_x,exactDate_y,format_postfix_x,format_postfix_y,format_prefix_x,format_prefix_y,isbn_x,isbn_y,ismn_x,ismn_y,masters_docid,musicid_x,musicid_y,pages_x,pages_y,part_x,part_y,person_100_x,person_100_y,person_245c_x,person_245c_y,person_700_x,person_700_y,pubinit_x,pubinit_y,pubword_x,pubword_y,pubyear_x,pubyear_y,scale_x,scale_y,ttlfull_245_x,ttlfull_245_y,ttlfull_246_x,ttlfull_246_y,ttlpart_x,ttlpart_y,volumes_x,volumes_y
0,"[(OCoLC)731635279, (ABN)000539983]","[(OCoLC)731635279, (ABN)000539983]",2009,2009,,,,,[],[],,,,,,,2009,2009,000311049,000311049,,,1,,,2009uuuu,2009uuuu,020000,020000,bk,bk,[978-3-15-020008-7],[978-3-15-020008-7],,,504389793,,,[600 S.],[600 S.],20008,20008,austenjane,austenjane,jane austen ; aus dem englischen übersetzt von...,jane austen ; aus dem englischen übersetzt von...,"grawechristian, graweursula","grawechristian, graweursula",reclam jun.,reclam jun.,[Reclam jun.],[Reclam jun.],2009,2009,,,"emma, roman","emma, roman",,,"{'245': ['Emma', 'Roman']}","{'245': ['Emma', 'Roman']}",600,600
1,"[(OCoLC)731635279, (ABN)000539983]","[(OCoLC)731635279, (NEBIS)009587153]",2009,2009,,,,,[],[],,,,,,,2009,2009,000311049,196506476,,,1,,,2009uuuu,2009uuuu,020000,020000,bk,bk,[978-3-15-020008-7],[978-3-15-020008-7],,,504389793,,,[600 S.],[600 S.],20008,20008,austenjane,austenjane,jane austen ; aus dem englischen übersetzt von...,jane austen ; aus dem engl. übers. von ursula ...,"grawechristian, graweursula",,reclam jun.,reclam,[Reclam jun.],[Reclam],2009,2009,,,"emma, roman",emma,,,"{'245': ['Emma', 'Roman']}",{'245': ['Emma']},600,600
2,"[(OCoLC)731635279, (ABN)000539983]","[(OCoLC)731635279, (LIBIB)000315536]",2009,2009,,,,,[],[],,,,,,,2009,2009,000311049,323173349,,,1,,,2009uuuu,2009uuuu,020000,020000,bk,bk,[978-3-15-020008-7],[978-3-15-020008-7],,,504389793,,,[600 S.],[600 S.],20008,20008,austenjane,austenjane,jane austen ; aus dem englischen übersetzt von...,jane austen,"grawechristian, graweursula",,reclam jun.,reclam,[Reclam jun.],[Reclam],2009,2009,,,"emma, roman","emma, roman",,,"{'245': ['Emma', 'Roman']}","{'245': ['Emma', 'Roman']}",600,600
3,"[(OCoLC)731635279, (NEBIS)009587153]","[(OCoLC)731635279, (ABN)000539983]",2009,2009,,,,,[],[],,,,,,,2009,2009,196506476,000311049,,,1,,,2009uuuu,2009uuuu,020000,020000,bk,bk,[978-3-15-020008-7],[978-3-15-020008-7],,,504389793,,,[600 S.],[600 S.],20008,20008,austenjane,austenjane,jane austen ; aus dem engl. übers. von ursula ...,jane austen ; aus dem englischen übersetzt von...,,"grawechristian, graweursula",reclam,reclam jun.,[Reclam],[Reclam jun.],2009,2009,,,emma,"emma, roman",,,{'245': ['Emma']},"{'245': ['Emma', 'Roman']}",600,600
4,"[(OCoLC)731635279, (NEBIS)009587153]","[(OCoLC)731635279, (NEBIS)009587153]",2009,2009,,,,,[],[],,,,,,,2009,2009,196506476,196506476,,,1,,,2009uuuu,2009uuuu,020000,020000,bk,bk,[978-3-15-020008-7],[978-3-15-020008-7],,,504389793,,,[600 S.],[600 S.],20008,20008,austenjane,austenjane,jane austen ; aus dem engl. übers. von ursula ...,jane austen ; aus dem engl. übers. von ursula ...,,,reclam,reclam,[Reclam],[Reclam],2009,2009,,,emma,emma,,,{'245': ['Emma']},{'245': ['Emma']},600,600


## Functions for Similarity Metrics Analysis

All available metrics algorithms of library [[TeDi](./A_References.ipynb#tedi)] are listed in the dictionary below. The dictionary will help calculating a similarity value for each available algorithm of the library.

In [2]:
import textdistance as tedi

tedi_algorithms = {
    # Edit based
    'Hamming' : tedi.Hamming(), 'MLIPNS' : tedi.MLIPNS(), 'Levenshtein' : tedi.Levenshtein(),
    'DamerauLevenshtein' : tedi.DamerauLevenshtein(), 'Jaro' : tedi.Jaro(), 'JaroWinkler' : tedi.JaroWinkler(),
    'StrCmp95' : tedi.StrCmp95(), 'NeedlemanWunsch' : tedi.NeedlemanWunsch(), 'Gotoh' : tedi.Gotoh(),
    'SmithWaterman' : tedi.SmithWaterman(),
    # Token based
    'Jaccard' : tedi.Jaccard(), 'Sorensen' : tedi.Sorensen(), 'Tversky' : tedi.Tversky(), 'Overlap' : tedi.Overlap(),
    'Tanimoto' : tedi.Tanimoto(), 'Cosine' : tedi.Cosine(), 'MongeElkan' : tedi.MongeElkan(), 'Bag' : tedi.Bag(),
    # Sequence based
    'LCSSeq' : tedi.LCSSeq(), 'LCSStr' : tedi.LCSStr(), 'RatcliffObershelp' : tedi.RatcliffObershelp(),
    # Compression based
    'ArithNCD' : tedi.ArithNCD(), 'RLENCD' : tedi.RLENCD(), 'BWTRLENCD' : tedi.BWTRLENCD(),
    'SqrtNCD' : tedi.SqrtNCD(), 'EntropyNCD' : tedi.EntropyNCD(), 'BZ2NCD' : tedi.BZ2NCD(),
    'LZMANCD' : tedi.LZMANCD(), 'ZLIBNCD' : tedi.ZLIBNCD(),
    # Phonetic
    'MRA' : tedi.MRA(), 'Editex' : tedi.Editex(),
    # Simple
    'Prefix' : tedi.Prefix(), 'Postfix' : tedi.Postfix(), 'Length' : tedi.Length(), 'Identity' : tedi.Identity(),
    'Matrix' : tedi.Matrix()
}

This appendix uses function $\texttt{.apply}\_\texttt{similarities()}$ that applies the $\texttt{.normalized}\_\texttt{similarity()}$ function of $\texttt{textdistance}$ for each algorithm object available in the library. The function is implemented in the separate code file [data_analysis_funcs.py](./data_analysis_funcs.py) 

In [3]:
import data_analysis_funcs as daf

## Similarity Metric Assessments

This section iterates through all available similarity metrics of library [[TeDi](./A_References.ipynb#tedi)] and calculates the similarity values for a pair of two sample strings of each feature of the model. The calculated similarity values will be analysed visually and an algorithm will be decided to be used in chapter [Feature Matrix Generation](./3_FeatureMatrixGeneration.ipynb). The decision will be based on a visual assessment that is validated with the literature, [[Chri2012](./A_References.ipynb#chri2012)].

In [4]:
# Extend display to number of columns of DataFrame
pd.options.display.max_columns = len(tedi_algorithms)+3

def num_of_samples (df) :
    max_number_of_num_samples = 30

    return min(len(df), max_number_of_num_samples)

### coordinate

In [5]:
df_string_pairs = daf.string_pair_list(df_feature_base, 'coordinate_E_x')

for algorithm in tedi_algorithms :
    if algorithm not in ['Gotoh', 'ArithNCD'] :
        daf.apply_similarities(df_string_pairs, tedi_algorithms[algorithm], algorithm)

df_string_pairs.sample(n=num_of_samples(df_string_pairs))

Hamming
MLIPNS
Levenshtein
DamerauLevenshtein
Jaro
JaroWinkler
StrCmp95
NeedlemanWunsch
SmithWaterman
Jaccard
Sorensen
Tversky
Overlap
Tanimoto
Cosine
MongeElkan
Bag
LCSSeq
LCSStr
RatcliffObershelp
RLENCD
BWTRLENCD
SqrtNCD
EntropyNCD
BZ2NCD
LZMANCD
ZLIBNCD
MRA
Editex
Prefix
Postfix
Length
Identity
Matrix


,str1,str2,Hamming,MLIPNS,Levenshtein,DamerauLevenshtein,Jaro,JaroWinkler,StrCmp95,NeedlemanWunsch,SmithWaterman,Jaccard,Sorensen,Tversky,Overlap,Tanimoto,Cosine,MongeElkan,Bag,LCSSeq,LCSStr,RatcliffObershelp,RLENCD,BWTRLENCD,SqrtNCD,EntropyNCD,BZ2NCD,LZMANCD,ZLIBNCD,MRA,Editex,Prefix,Postfix,Length,Identity,Matrix
1,e0055009,e0055700,0.750,1.0,0.750,0.750,0.916667,0.950000,0.925000,0.8750,0.750,0.777778,0.875,0.777778,0.875,-0.362570,0.875,0.054688,0.875,0.875,0.625,0.875,0.0,0.333333,0.477592,0.954545,0.896552,0.84,0.642857,0.600000,0.750,0.625,0.0,1.0,0.0,0.0
8,e0055009,e0075850,0.500,0.0,0.500,0.500,0.833333,0.883333,0.850000,0.7500,0.500,0.600000,0.750,0.600000,0.750,-0.736966,0.750,0.054688,0.750,0.750,0.375,0.750,0.0,0.222222,0.406835,0.874262,0.862069,0.84,0.428571,0.333333,0.625,0.375,0.0,1.0,0.0,0.0
5,e0055009,e0074147,0.375,0.0,0.375,0.375,0.583333,0.583333,0.583333,0.6875,0.375,0.230769,0.375,0.230769,0.375,-2.115477,0.375,0.039062,0.375,0.375,0.375,0.375,0.0,0.222222,0.248374,0.759803,0.862069,0.84,0.428571,0.333333,0.375,0.375,0.0,1.0,0.0,0.0
7,e0055009,e0080900,0.500,0.0,0.500,0.625,0.777778,0.844444,0.800000,0.7500,0.500,0.600000,0.750,0.600000,0.750,-0.736966,0.750,0.046875,0.750,0.625,0.375,0.625,0.0,0.555556,0.444689,0.893410,0.896552,0.84,0.428571,0.666667,0.750,0.375,0.0,1.0,0.0,0.0
0,e0055009,,0.000,0.0,0.000,0.000,0.000000,0.000000,0.000000,0.0000,0.000,0.000000,0.000,0.000000,0.000,-inf,0.000,0.000000,0.000,0.000,0.000,0.000,0.0,0.111111,0.000000,0.363636,0.000000,0.36,0.428571,0.000000,0.000,0.000,0.0,0.0,0.0,0.0
4,e0055009,e0060811,0.375,0.0,0.375,0.500,0.666667,0.666667,0.666667,0.6875,0.375,0.333333,0.500,0.333333,0.500,-1.584963,0.500,0.039062,0.500,0.500,0.375,0.500,0.0,0.333333,0.272049,0.795039,0.862069,0.84,0.428571,0.500000,0.625,0.375,0.0,1.0,0.0,0.0
2,e0055009,e0080855,0.375,0.0,0.375,0.500,0.777778,0.844444,0.800000,0.6875,0.375,0.600000,0.750,0.600000,0.750,-0.736966,0.750,0.054688,0.750,0.625,0.375,0.625,0.0,0.333333,0.449694,0.906464,0.896552,0.84,0.428571,0.500000,0.625,0.375,0.0,1.0,0.0,0.0
3,e0055009,e0080851,0.375,0.0,0.375,0.500,0.683333,0.683333,0.683333,0.6875,0.375,0.454545,0.625,0.454545,0.625,-1.137504,0.625,0.054688,0.625,0.500,0.375,0.500,0.0,0.333333,0.383037,0.849602,0.862069,0.84,0.428571,0.333333,0.500,0.375,0.0,1.0,0.0,0.0
6,e0055009,e0055009,1.000,1.0,1.000,1.000,1.000000,1.000000,1.000000,1.0000,1.000,1.000000,1.000,1.000000,1.000,0.000000,1.000,1.000000,1.000,1.000,1.000,1.000,0.0,0.555556,0.585786,1.000000,0.965517,0.84,0.857143,1.000000,1.000,1.000,1.0,1.0,1.0,1.0


### corporate

In [6]:
df_string_pairs = daf.string_pair_list(df_feature_base, 'corporate_full_x')

for algorithm in tedi_algorithms :
    if algorithm not in ['Gotoh', 'ArithNCD'] :
        daf.apply_similarities(df_string_pairs, tedi_algorithms[algorithm], algorithm)

df_string_pairs.sample(n=num_of_samples(df_string_pairs))

Hamming
MLIPNS
Levenshtein
DamerauLevenshtein
Jaro
JaroWinkler
StrCmp95
NeedlemanWunsch
SmithWaterman
Jaccard
Sorensen
Tversky
Overlap
Tanimoto
Cosine
MongeElkan
Bag
LCSSeq
LCSStr
RatcliffObershelp
RLENCD
BWTRLENCD
SqrtNCD
EntropyNCD
BZ2NCD
LZMANCD
ZLIBNCD
MRA
Editex
Prefix
Postfix
Length
Identity
Matrix


,str1,str2,Hamming,MLIPNS,Levenshtein,DamerauLevenshtein,Jaro,JaroWinkler,StrCmp95,NeedlemanWunsch,SmithWaterman,Jaccard,Sorensen,Tversky,Overlap,Tanimoto,Cosine,MongeElkan,Bag,LCSSeq,LCSStr,RatcliffObershelp,RLENCD,BWTRLENCD,SqrtNCD,EntropyNCD,BZ2NCD,LZMANCD,ZLIBNCD,MRA,Editex,Prefix,Postfix,Length,Identity,Matrix
35,schweiz,schweizerische normen-vereinigung,0.212121,0.0,0.212121,0.212121,0.737374,0.842424,0.842424,0.212121,0.000000,0.212121,0.350000,0.212121,1.000000,-2.237039,0.460566,0.071429,0.212121,0.212121,0.212121,0.350000,0.0,0.088235,0.219143,0.807028,0.540000,0.538462,0.307692,0.500000,0.272727,0.212121,0.0,0.212121,0.0,0.0
61,schweiz,philadelphia orchestra,0.000000,0.0,0.090909,0.136364,0.410534,0.410534,0.429365,0.204545,0.142857,0.208333,0.344828,0.208333,0.714286,-2.263034,0.402911,0.051020,0.227273,0.136364,0.090909,0.206897,0.0,0.043478,0.197636,0.794855,0.604651,0.636364,0.285714,0.000000,0.204545,0.000000,0.0,0.318182,0.0,0.0
48,schweiz,bühnen kölnoper,0.133333,0.0,0.133333,0.133333,0.473016,0.473016,0.514921,0.300000,0.000000,0.100000,0.181818,0.100000,0.285714,-3.321928,0.195180,0.020408,0.133333,0.133333,0.066667,0.181818,0.0,0.062500,0.096449,0.746687,0.500000,0.741935,0.260870,0.166667,0.166667,0.000000,0.0,0.466667,0.0,0.0
62,schweiz,"bruder-klausen-stiftung, trägerverein 600 jahr...",0.014493,0.0,0.057971,0.057971,0.379572,0.379572,0.426777,0.079710,0.000000,0.055556,0.105263,0.055556,0.571429,-4.169925,0.182006,0.040816,0.057971,0.057971,0.028986,0.078947,0.0,0.014286,0.069189,0.673687,0.258427,0.368421,0.081081,0.000000,0.108696,0.000000,0.0,0.101449,0.0,0.0
0,schweiz,,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-inf,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.125000,0.000000,0.262650,0.000000,0.360000,0.461538,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0
63,schweiz,historischer verein des kantons solothurn,0.000000,0.0,0.121951,0.121951,0.478746,0.478746,0.512195,0.146341,0.000000,0.116279,0.208333,0.116279,0.714286,-3.104337,0.295141,0.051020,0.121951,0.121951,0.073171,0.208333,0.0,0.023810,0.147363,0.774379,0.388889,0.534884,0.177778,0.000000,0.170732,0.000000,0.0,0.170732,0.0,0.0
52,schweiz,"interkantonale lehrmittelzentrale, staatlicher...",0.000000,0.0,0.071429,0.071429,0.459524,0.459524,0.490952,0.085714,0.000000,0.084507,0.155844,0.084507,0.857143,-3.564785,0.271052,0.061224,0.085714,0.071429,0.028571,0.129870,0.0,0.014706,0.118589,0.762772,0.337838,0.418182,0.126984,0.000000,0.142857,0.000000,0.0,0.100000,0.0,0.0
51,schweiz,"husner, kloster rheinau",0.043478,0.0,0.130435,0.130435,0.408558,0.408558,0.483092,0.217391,0.000000,0.153846,0.266667,0.153846,0.571429,-2.700440,0.315244,0.040816,0.173913,0.173913,0.086957,0.266667,0.0,0.041667,0.152959,0.766708,0.558140,0.636364,0.206897,0.000000,0.217391,0.000000,0.0,0.304348,0.0,0.0
10,schweiz,the metropolitan chorus und orchestra,0.000000,0.0,0.081081,0.108108,0.393179,0.393179,0.427156,0.135135,0.142857,0.128205,0.227273,0.128205,0.714286,-2.963474,0.310685,0.051020,0.135135,0.108108,0.054054,0.136364,0.0,0.052632,0.145069,0.758533,0.461538,0.560976,0.186047,0.000000,0.135135,0.000000,0.0,0.189189,0.0,0.0
7,schweiz,schweizerische gesellschaft für bildungsforsch...,0.070000,0.0,0.070000,0.070000,0.690000,0.690000,0.690000,0.070000,0.285714,0.070000,0.130841,0.070000,1.000000,-3.836501,0.264575,0.071429,0.070000,0.070000,0.070000,0.130841,0.0,0.078652,0.126911,0.751983,0.241758,0.396825,0.146341,0.500000,0.185000,0.070000,0.0,0.070000,0.0,0.0


Monge-Elkan, Jaccard with q-grams, and LCSStr seem to be valid metrics for the $\texttt{corporate}$ attribute due to their algorithms [[Chri2012](./A_References.ipynb#chri2012)]. The metrics to be chosen will be analysed and justifyed in chapter [Feature Matrix Generation](./4_FeatureMatrixGeneration.ipynb).

### doi

For attribute $\texttt{doi}$, a preprocessing function has been implemented to extract real doi identifiers, see chapter [Data Analysis](./1_DataAnalysis.ipynb). The DataFrame $\texttt{df}\_\texttt{feature}\_\texttt{base}$ holds doi identifiers as preprocessed singular strings.

In [7]:
df_string_pairs = daf.string_pair_list(df_feature_base, 'doi_x')

for algorithm in tedi_algorithms :
    if algorithm not in ['Gotoh', 'ArithNCD'] :
        daf.apply_similarities(df_string_pairs, tedi_algorithms[algorithm], algorithm)

df_string_pairs.sample(n=num_of_samples(df_string_pairs))

Hamming
MLIPNS
Levenshtein
DamerauLevenshtein
Jaro
JaroWinkler
StrCmp95
NeedlemanWunsch
SmithWaterman
Jaccard
Sorensen
Tversky
Overlap
Tanimoto
Cosine
MongeElkan
Bag
LCSSeq
LCSStr
RatcliffObershelp
RLENCD
BWTRLENCD
SqrtNCD
EntropyNCD
BZ2NCD
LZMANCD
ZLIBNCD
MRA
Editex
Prefix
Postfix
Length
Identity
Matrix


,str1,str2,Hamming,MLIPNS,Levenshtein,DamerauLevenshtein,Jaro,JaroWinkler,StrCmp95,NeedlemanWunsch,SmithWaterman,Jaccard,Sorensen,Tversky,Overlap,Tanimoto,Cosine,MongeElkan,Bag,LCSSeq,LCSStr,RatcliffObershelp,RLENCD,BWTRLENCD,SqrtNCD,EntropyNCD,BZ2NCD,LZMANCD,ZLIBNCD,MRA,Editex,Prefix,Postfix,Length,Identity,Matrix
51,10.5167/uzh-57152,10.3931/e-rara-50903,0.250000,0.0,0.300000,0.300000,0.587255,0.587255,0.598137,0.575000,0.176471,0.233333,0.378378,0.233333,0.411765,-2.099536,0.379628,0.017301,0.350000,0.350000,0.150000,0.378378,0.0,0.047619,0.290883,0.878591,0.666667,0.677419,0.230769,0.500000,0.325000,0.150000,0.000000,0.850000,0.0,0.0
32,10.5167/uzh-57152,10.5169/seals-377028,0.350000,0.0,0.500000,0.550000,0.702050,0.821230,0.745579,0.675000,0.411765,0.423077,0.594595,0.423077,0.647059,-1.241008,0.596559,0.024221,0.550000,0.500000,0.300000,0.486486,0.0,0.095238,0.344442,0.878098,0.659091,0.677419,0.346154,0.500000,0.550000,0.300000,0.000000,0.850000,0.0,0.0
16,10.5167/uzh-57152,10.5169/seals-376437,0.350000,0.0,0.450000,0.450000,0.662745,0.662745,0.706275,0.650000,0.352941,0.370370,0.540541,0.370370,0.588235,-1.432959,0.542326,0.022491,0.500000,0.450000,0.300000,0.432432,0.0,0.095238,0.321947,0.875570,0.651163,0.677419,0.346154,0.500000,0.500000,0.300000,0.000000,0.850000,0.0,0.0
9,10.5167/uzh-57152,10.1055/b-005-143650,0.300000,0.0,0.350000,0.400000,0.596078,0.596078,0.596078,0.600000,0.235294,0.423077,0.594595,0.423077,0.647059,-1.241008,0.596559,0.019031,0.550000,0.450000,0.150000,0.378378,0.0,0.047619,0.383837,0.888551,0.634146,0.677419,0.269231,0.666667,0.425000,0.150000,0.000000,0.850000,0.0,0.0
49,10.5167/uzh-57152,10.5169/seals-515356,0.350000,0.0,0.500000,0.550000,0.732353,0.839412,0.754118,0.675000,0.411765,0.423077,0.594595,0.423077,0.647059,-1.241008,0.596559,0.019031,0.550000,0.550000,0.300000,0.594595,0.0,0.150000,0.329438,0.911713,0.704545,0.677419,0.346154,0.666667,0.550000,0.300000,0.000000,0.850000,0.0,0.0
40,10.5167/uzh-57152,10.5169/seals-377305,0.350000,0.0,0.450000,0.500000,0.671747,0.671747,0.715276,0.650000,0.352941,0.423077,0.594595,0.423077,0.647059,-1.241008,0.596559,0.022491,0.550000,0.500000,0.300000,0.432432,0.0,0.095238,0.344194,0.893595,0.619048,0.677419,0.346154,0.500000,0.550000,0.300000,0.000000,0.850000,0.0,0.0
28,10.5167/uzh-57152,10.5169/seals-376890,0.350000,0.0,0.450000,0.450000,0.622767,0.622767,0.666296,0.650000,0.352941,0.321429,0.486486,0.321429,0.529412,-1.637430,0.488094,0.022491,0.450000,0.450000,0.300000,0.378378,0.0,0.095238,0.319025,0.872822,0.659091,0.677419,0.346154,0.500000,0.500000,0.300000,0.000000,0.850000,0.0,0.0
46,10.5167/uzh-57152,10.5169/seals-515321,0.350000,0.0,0.550000,0.550000,0.768627,0.861176,0.790392,0.675000,0.411765,0.480000,0.648649,0.480000,0.705882,-1.058894,0.650791,0.020761,0.600000,0.600000,0.300000,0.648649,0.0,0.190476,0.347797,0.909188,0.697674,0.677419,0.346154,0.500000,0.600000,0.300000,0.000000,0.850000,0.0,0.0
20,10.5167/uzh-57152,10.5169/seals-376572,0.350000,0.0,0.550000,0.550000,0.740850,0.844510,0.784379,0.700000,0.470588,0.480000,0.648649,0.480000,0.705882,-1.058894,0.650791,0.024221,0.600000,0.550000,0.300000,0.540541,0.0,0.095238,0.366697,0.895802,0.651163,0.677419,0.346154,0.666667,0.600000,0.300000,0.050000,0.850000,0.0,0.0
3,10.5167/uzh-57152,10.1055/b-002-26639,0.210526,0.0,0.263158,0.315789,0.593625,0.593625,0.604770,0.578947,0.176471,0.384615,0.555556,0.384615,0.588235,-1.378512,0.556415,0.020761,0.526316,0.368421,0.157895,0.277778,0.0,0.050000,0.412363,0.923139,0.731707,0.741935,0.240000,0.500000,0.368421,0.157895,0.000000,0.894737,0.0,0.0


### edition

In [8]:
df_string_pairs = daf.string_pair_list(df_feature_base, 'edition_x')

for algorithm in tedi_algorithms :
    if algorithm not in ['Gotoh', 'ArithNCD'] :
        daf.apply_similarities(df_string_pairs, tedi_algorithms[algorithm], algorithm)

df_string_pairs.sample(n=num_of_samples(df_string_pairs))

Hamming
MLIPNS
Levenshtein
DamerauLevenshtein
Jaro
JaroWinkler
StrCmp95
NeedlemanWunsch
SmithWaterman
Jaccard
Sorensen
Tversky
Overlap
Tanimoto
Cosine
MongeElkan
Bag
LCSSeq
LCSStr
RatcliffObershelp
RLENCD
BWTRLENCD
SqrtNCD
EntropyNCD
BZ2NCD
LZMANCD
ZLIBNCD
MRA
Editex
Prefix
Postfix
Length
Identity
Matrix


,str1,str2,Hamming,MLIPNS,Levenshtein,DamerauLevenshtein,Jaro,JaroWinkler,StrCmp95,NeedlemanWunsch,SmithWaterman,Jaccard,Sorensen,Tversky,Overlap,Tanimoto,Cosine,MongeElkan,Bag,LCSSeq,LCSStr,RatcliffObershelp,RLENCD,BWTRLENCD,SqrtNCD,EntropyNCD,BZ2NCD,LZMANCD,ZLIBNCD,MRA,Editex,Prefix,Postfix,Length,Identity,Matrix
14,6,2007,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.125000,0.0,0.00,0.0,0.00,0.0,-inf,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.200000,0.000000e+00,0.231229,0.875000,0.913043,0.600000,0.0,0.250000,0.0,0.00,0.250000,0.0,0.0
6,6,19,0.0,1.0,0.00,0.00,0.0,0.0,0.0,0.250000,0.0,0.00,0.0,0.00,0.0,-inf,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.333333,0.000000e+00,0.207519,0.954545,1.000000,0.750000,0.0,0.000000,0.0,0.00,0.500000,0.0,0.0
19,6,1889,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.125000,0.0,0.00,0.0,0.00,0.0,-inf,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.200000,0.000000e+00,0.231229,0.826087,0.913043,0.600000,0.0,0.250000,0.0,0.00,0.250000,0.0,0.0
24,6,1926,0.0,0.0,0.25,0.25,0.0,0.0,0.0,0.250000,1.0,0.25,0.4,0.25,1.0,-2.0,0.5,0.5,0.25,0.25,0.25,0.4,0.0,0.200000,1.464466e-01,0.359357,0.880000,0.913043,0.600000,0.0,0.250000,0.0,0.25,0.250000,0.0,0.0
28,6,2000,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.125000,0.0,0.00,0.0,0.00,0.0,-inf,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.250000,0.000000e+00,0.243103,0.909091,0.913043,0.600000,0.0,0.500000,0.0,0.00,0.250000,0.0,0.0
22,6,1907,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.125000,0.0,0.00,0.0,0.00,0.0,-inf,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.200000,0.000000e+00,0.226024,0.880000,0.913043,0.600000,0.0,0.000000,0.0,0.00,0.250000,0.0,0.0
17,6,1869,0.0,0.0,0.25,0.25,0.0,0.0,0.0,0.250000,0.0,0.25,0.4,0.25,1.0,-2.0,0.5,0.5,0.25,0.25,0.25,0.4,0.0,0.200000,1.464466e-01,0.359357,0.880000,0.913043,0.600000,0.0,0.250000,0.0,0.00,0.250000,0.0,0.0
9,6,4,0.0,1.0,0.00,0.00,0.0,0.0,0.0,0.500000,0.0,0.00,0.0,0.00,0.0,-inf,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.500000,0.000000e+00,0.000000,1.000000,1.000000,0.857143,0.0,0.000000,0.0,0.00,1.000000,0.0,0.0
0,6,,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0,-inf,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.500000,0.000000e+00,1.000000,0.000000,0.391304,0.857143,0.0,0.000000,0.0,0.00,0.000000,0.0,0.0
26,6,1994,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.125000,0.0,0.00,0.0,0.00,0.0,-inf,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.200000,0.000000e+00,0.231229,0.869565,0.913043,0.600000,0.0,0.250000,0.0,0.00,0.250000,0.0,0.0


### exactDate

In [9]:
df_string_pairs = daf.string_pair_list(df_feature_base, 'exactDate_x')

for algorithm in tedi_algorithms :
    if algorithm not in ['ArithNCD'] :
        daf.apply_similarities(df_string_pairs, tedi_algorithms[algorithm], algorithm)

df_string_pairs.sample(n=num_of_samples(df_string_pairs))

Hamming
MLIPNS
Levenshtein
DamerauLevenshtein
Jaro
JaroWinkler
StrCmp95
NeedlemanWunsch
Gotoh
SmithWaterman
Jaccard
Sorensen
Tversky
Overlap
Tanimoto
Cosine
MongeElkan
Bag
LCSSeq
LCSStr
RatcliffObershelp
RLENCD
BWTRLENCD
SqrtNCD
EntropyNCD
BZ2NCD
LZMANCD
ZLIBNCD
MRA
Editex
Prefix
Postfix
Length
Identity
Matrix


,str1,str2,Hamming,MLIPNS,Levenshtein,DamerauLevenshtein,Jaro,JaroWinkler,StrCmp95,NeedlemanWunsch,Gotoh,SmithWaterman,Jaccard,Sorensen,Tversky,Overlap,Tanimoto,Cosine,MongeElkan,Bag,LCSSeq,LCSStr,RatcliffObershelp,RLENCD,BWTRLENCD,SqrtNCD,EntropyNCD,BZ2NCD,LZMANCD,ZLIBNCD,MRA,Editex,Prefix,Postfix,Length,Identity,Matrix
175,1795uuuu,19961997,0.250,0.0,0.250,0.375,0.500000,0.500000,0.500000,0.6250,0.6250,0.250,0.230769,0.375,0.230769,0.375,-2.115477,0.375,0.023438,0.375,0.250,0.125,0.250,0.000,0.000000,0.338647,0.799261,0.733333,0.84,0.500000,0.166667,0.375,0.125,0.000,1.0,0.0,0.0
88,1795uuuu,20150201,0.125,0.0,0.125,0.125,0.500000,0.500000,0.500000,0.5625,0.5625,0.125,0.142857,0.250,0.142857,0.250,-2.807355,0.250,0.015625,0.250,0.250,0.125,0.250,0.000,0.000000,0.211325,0.750000,0.666667,0.84,0.500000,0.000000,0.125,0.000,0.000,1.0,0.0,0.0
112,1795uuuu,1473uuuu,0.625,1.0,0.625,0.625,0.833333,0.850000,0.833333,0.8125,0.8125,0.625,0.600000,0.750,0.600000,0.750,-0.736966,0.750,0.046875,0.750,0.750,0.500,0.750,0.000,0.375000,0.390524,0.916667,0.833333,0.84,0.500000,0.250000,0.625,0.125,0.500,1.0,0.0,0.0
160,1795uuuu,1490uuuu,0.750,1.0,0.750,0.750,0.833333,0.850000,0.833333,0.8750,0.8750,0.750,0.600000,0.750,0.600000,0.750,-0.736966,0.750,0.046875,0.750,0.750,0.500,0.750,0.000,0.375000,0.390524,0.916667,0.833333,0.84,0.500000,0.500000,0.750,0.125,0.500,1.0,0.0,0.0
117,1795uuuu,1953uuuu,0.625,1.0,0.750,0.750,0.916667,0.925000,0.916667,0.8125,0.8125,0.625,0.777778,0.875,0.777778,0.875,-0.362570,0.875,0.054688,0.875,0.875,0.500,0.875,0.000,0.375000,0.488155,0.958333,0.866667,0.84,0.500000,0.250000,0.750,0.125,0.500,1.0,0.0,0.0
138,1795uuuu,1844uuuu,0.625,1.0,0.625,0.625,0.750000,0.775000,0.750000,0.8125,0.8125,0.625,0.454545,0.625,0.454545,0.625,-1.137504,0.625,0.039062,0.625,0.625,0.500,0.625,0.000,0.375000,0.292893,0.833333,0.838710,0.84,0.500000,0.250000,0.625,0.125,0.500,1.0,0.0,0.0
102,1795uuuu,189uuuuu,0.750,1.0,0.750,0.875,0.833333,0.850000,0.833333,0.8750,0.8750,0.750,0.600000,0.750,0.600000,0.750,-0.736966,0.750,0.046875,0.750,0.750,0.500,0.750,0.000,0.375000,0.401273,0.860626,0.833333,0.84,0.500000,0.500000,0.750,0.125,0.500,1.0,0.0,0.0
182,1795uuuu,19282011,0.125,0.0,0.125,0.250,0.500000,0.500000,0.500000,0.5625,0.5625,0.125,0.142857,0.250,0.142857,0.250,-2.807355,0.250,0.015625,0.250,0.250,0.125,0.250,0.125,0.111111,0.214413,0.762330,0.666667,0.84,0.428571,0.166667,0.250,0.125,0.000,1.0,0.0,0.0
210,1795uuuu,1922uuuu,0.625,1.0,0.625,0.750,0.833333,0.850000,0.833333,0.8125,0.8125,0.625,0.600000,0.750,0.600000,0.750,-0.736966,0.750,0.046875,0.750,0.750,0.500,0.750,0.000,0.375000,0.390524,0.875000,0.870968,0.84,0.500000,0.250000,0.750,0.125,0.500,1.0,0.0,0.0
71,1795uuuu,1978uuuu,0.625,1.0,0.625,0.750,0.869048,0.882143,0.869048,0.8125,0.8125,0.625,0.777778,0.875,0.777778,0.875,-0.362570,0.875,0.054688,0.875,0.750,0.500,0.750,0.000,0.375000,0.488155,0.958333,0.866667,0.84,0.500000,0.250000,0.625,0.125,0.500,1.0,0.0,0.0


Attribute $\texttt{exactDate}$ is a string of four digits or characters. For calculating the [Hamming distance](https://en.wikipedia.org/wiki/Hamming_distance), each string pair is compared character-wise. A so called edit distance between the string pair is calculated as the sum of all edit operations needed to convert the strings into each other, [[Chri2012](./A_References.ipynb#chri2012)]. The resulting Hamming similarity can be deduced from the edit distance and the length of one string. This can be easily done in the examples of the DataFrame above. The Hamming similarity shall be used for attribute $\texttt{exactDate}$.

The Hamming similarity has one drawbak, though, looking at Swissbib's data. The attribute may be filled with letter 'u' for 'unknown' digits instead of a number. A letter 'u' will result in an edit distance of 1. This is a statement which need not be true for the bibliographic units that the records describe. On the other hand, a pair of strings with a letter 'u' at the same digit, need not have a distance of 0 for the two bibliographical units compared. For this reason, the Hamming similarity will be adapted for the case of existence of letter 'u' in one of the strings of the pair. The Hamming similarity will be increased by a small value for each unknown digit in a string. This algorithm based on the Hamming similarity will be explicitly implemented in chapter [Feature Matrix Generation](./4_FeatureMatrixGeneration.ipynb).

### format

In [10]:
df_string_pairs = daf.string_pair_list(df_feature_base, 'format_prefix_x')

for algorithm in tedi_algorithms :
    if algorithm not in [] :
        daf.apply_similarities(df_string_pairs, tedi_algorithms[algorithm], algorithm)

df_string_pairs.sample(n=num_of_samples(df_string_pairs))

Hamming
MLIPNS
Levenshtein
DamerauLevenshtein
Jaro
JaroWinkler
StrCmp95
NeedlemanWunsch
Gotoh
SmithWaterman
Jaccard
Sorensen
Tversky
Overlap
Tanimoto
Cosine
MongeElkan
Bag
LCSSeq
LCSStr
RatcliffObershelp
ArithNCD
RLENCD
BWTRLENCD
SqrtNCD
EntropyNCD
BZ2NCD
LZMANCD
ZLIBNCD
MRA
Editex
Prefix
Postfix
Length
Identity
Matrix


,str1,str2,Hamming,MLIPNS,Levenshtein,DamerauLevenshtein,Jaro,JaroWinkler,StrCmp95,NeedlemanWunsch,Gotoh,SmithWaterman,Jaccard,Sorensen,Tversky,Overlap,Tanimoto,Cosine,MongeElkan,Bag,LCSSeq,LCSStr,RatcliffObershelp,ArithNCD,RLENCD,BWTRLENCD,SqrtNCD,EntropyNCD,BZ2NCD,LZMANCD,ZLIBNCD,MRA,Editex,Prefix,Postfix,Length,Identity,Matrix
5,cf,mp,0.0,1.0,0.0,0.0,0.000000,0.000000,0.000000,0.50,0.50,0.0,0.000000,0.0,0.000000,0.0,-inf,0.0,0.000,0.0,0.0,0.0,0.0,-0.666667,0.0,0.333333,0.000000,0.50,0.791667,1.0,0.75,0.0,0.25,0.0,0.0,1.0,0.0,0.0
1,cf,vm,0.0,1.0,0.0,0.0,0.000000,0.000000,0.000000,0.50,0.50,0.0,0.000000,0.0,0.000000,0.0,-inf,0.0,0.000,0.0,0.0,0.0,0.0,-0.333333,0.0,0.333333,0.000000,0.50,0.791667,1.0,0.75,0.0,0.00,0.0,0.0,1.0,0.0,0.0
2,cf,mu,0.0,1.0,0.0,0.0,0.000000,0.000000,0.000000,0.50,0.50,0.0,0.000000,0.0,0.000000,0.0,-inf,0.0,0.000,0.0,0.0,0.0,0.0,-0.666667,0.0,0.333333,0.000000,0.50,0.791667,1.0,0.75,0.0,0.00,0.0,0.0,1.0,0.0,0.0
4,cf,cf,1.0,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.00,1.00,1.0,1.000000,1.0,1.000000,1.0,0.000000,1.0,1.000,1.0,1.0,1.0,1.0,0.333333,0.0,0.333333,0.585786,1.00,1.000000,1.0,0.75,1.0,1.00,1.0,1.0,1.0,1.0,1.0
3,cf,cr,0.5,1.0,0.5,0.5,0.666667,0.666667,0.666667,0.75,0.75,0.5,0.333333,0.5,0.333333,0.5,-1.584963,0.5,0.125,0.5,0.5,0.5,0.5,0.000000,0.0,0.333333,0.292893,0.75,0.833333,1.0,0.75,0.5,0.50,0.5,0.0,1.0,0.0,0.0
0,cf,bk,0.0,1.0,0.0,0.0,0.000000,0.000000,0.000000,0.50,0.50,0.0,0.000000,0.0,0.000000,0.0,-inf,0.0,0.000,0.0,0.0,0.0,0.0,-1.000000,0.0,0.333333,0.000000,0.50,0.863636,1.0,0.75,0.0,0.00,0.0,0.0,1.0,0.0,0.0


In [11]:
df_string_pairs = daf.string_pair_list(df_feature_base, 'format_postfix_x')

for algorithm in tedi_algorithms :
    if algorithm not in [] :
        daf.apply_similarities(df_string_pairs, tedi_algorithms[algorithm], algorithm)

df_string_pairs.sample(n=num_of_samples(df_string_pairs))

Hamming
MLIPNS
Levenshtein
DamerauLevenshtein
Jaro
JaroWinkler
StrCmp95
NeedlemanWunsch
Gotoh
SmithWaterman
Jaccard
Sorensen
Tversky
Overlap
Tanimoto
Cosine
MongeElkan
Bag
LCSSeq
LCSStr
RatcliffObershelp
ArithNCD
RLENCD
BWTRLENCD
SqrtNCD
EntropyNCD
BZ2NCD
LZMANCD
ZLIBNCD
MRA
Editex
Prefix
Postfix
Length
Identity
Matrix


,str1,str2,Hamming,MLIPNS,Levenshtein,DamerauLevenshtein,Jaro,JaroWinkler,StrCmp95,NeedlemanWunsch,Gotoh,SmithWaterman,Jaccard,Sorensen,Tversky,Overlap,Tanimoto,Cosine,MongeElkan,Bag,LCSSeq,LCSStr,RatcliffObershelp,ArithNCD,RLENCD,BWTRLENCD,SqrtNCD,EntropyNCD,BZ2NCD,LZMANCD,ZLIBNCD,MRA,Editex,Prefix,Postfix,Length,Identity,Matrix
9,010200,020053,0.333333,0.0,0.333333,0.500000,0.694444,0.694444,0.694444,0.666667,0.666667,0.333333,0.500000,0.666667,0.500000,0.666667,-1.000000,0.666667,0.069444,0.666667,0.666667,0.666667,0.666667,-1.500000,0.166667,0.571429,0.353353,0.810547,0.880000,0.92,0.750000,0.400000,0.500000,0.166667,0.000000,1.0,0.0,0.0
14,010200,020800,0.666667,1.0,0.666667,0.666667,0.822222,0.840000,0.822222,0.833333,0.833333,0.666667,0.714286,0.833333,0.714286,0.833333,-0.485427,0.833333,0.069444,0.833333,0.666667,0.500000,0.666667,-0.666667,0.166667,0.428571,0.439340,0.925980,0.875000,0.92,0.666667,0.600000,0.666667,0.166667,0.333333,1.0,0.0,0.0
21,010200,020400,0.666667,1.0,0.666667,0.666667,0.822222,0.840000,0.822222,0.833333,0.833333,0.666667,0.714286,0.833333,0.714286,0.833333,-0.485427,0.833333,0.069444,0.833333,0.666667,0.500000,0.666667,-0.666667,0.166667,0.428571,0.439340,0.925980,0.875000,0.92,0.666667,0.600000,0.666667,0.166667,0.333333,1.0,0.0,0.0
17,010200,040000,0.666667,1.0,0.666667,0.833333,0.777778,0.800000,0.777778,0.833333,0.833333,0.666667,0.500000,0.666667,0.500000,0.666667,-1.000000,0.666667,0.055556,0.666667,0.666667,0.333333,0.500000,-0.666667,0.166667,0.428571,0.309017,0.752403,0.769231,0.92,0.583333,0.400000,0.666667,0.166667,0.333333,1.0,0.0,0.0
25,010200,020347,0.333333,0.0,0.333333,0.333333,0.555556,0.555556,0.555556,0.666667,0.666667,0.333333,0.333333,0.500000,0.333333,0.500000,-1.584963,0.500000,0.069444,0.500000,0.500000,0.500000,0.500000,-0.416667,0.166667,0.428571,0.286378,0.731155,0.880000,0.92,0.666667,0.333333,0.333333,0.166667,0.000000,1.0,0.0,0.0
13,010200,030500,0.666667,1.0,0.666667,0.666667,0.777778,0.800000,0.777778,0.833333,0.833333,0.666667,0.500000,0.666667,0.500000,0.666667,-1.000000,0.666667,0.055556,0.666667,0.666667,0.333333,0.666667,-0.333333,0.166667,0.428571,0.292893,0.851959,0.875000,0.92,0.500000,0.600000,0.666667,0.166667,0.333333,1.0,0.0,0.0
8,010200,020047,0.333333,0.0,0.333333,0.500000,0.694444,0.694444,0.694444,0.666667,0.666667,0.333333,0.500000,0.666667,0.500000,0.666667,-1.000000,0.666667,0.069444,0.666667,0.666667,0.666667,0.666667,0.000000,0.166667,0.571429,0.353353,0.810547,0.923077,0.92,0.750000,0.400000,0.500000,0.166667,0.000000,1.0,0.0,0.0
19,010200,030053,0.333333,0.0,0.333333,0.500000,0.666667,0.666667,0.666667,0.666667,0.666667,0.333333,0.333333,0.500000,0.333333,0.500000,-1.584963,0.500000,0.055556,0.500000,0.500000,0.333333,0.500000,-1.500000,0.166667,0.428571,0.261995,0.784867,0.880000,0.92,0.500000,0.400000,0.500000,0.166667,0.000000,1.0,0.0,0.0
11,010200,030700,0.666667,1.0,0.666667,0.666667,0.777778,0.800000,0.777778,0.833333,0.833333,0.666667,0.500000,0.666667,0.500000,0.666667,-1.000000,0.666667,0.055556,0.666667,0.666667,0.333333,0.666667,-0.333333,0.166667,0.428571,0.292893,0.851959,0.875000,0.92,0.500000,0.600000,0.666667,0.166667,0.333333,1.0,0.0,0.0
5,010200,010100,0.833333,1.0,0.833333,0.833333,0.888889,0.922222,0.888889,0.916667,0.916667,0.833333,0.714286,0.833333,0.714286,0.833333,-0.485427,0.833333,0.069444,0.833333,0.833333,0.500000,0.833333,-0.500000,0.166667,0.428571,0.463434,0.879898,0.875000,0.92,0.666667,0.800000,0.833333,0.500000,0.333333,1.0,0.0,0.0


### isbn

Attribute $\texttt{isbn}$ is treated as a list of string elements, see chapter [Feature Matrix Generation](./4_FeatureMatrixGeneration.ipynb). The metrics comparison will be ommitted, here.

### ismn

In [12]:
df_string_pairs = daf.string_pair_list(df_feature_base, 'ismn_x')

for algorithm in tedi_algorithms :
    if algorithm not in ['Gotoh', 'ArithNCD'] :
        daf.apply_similarities(df_string_pairs, tedi_algorithms[algorithm], algorithm)

df_string_pairs.sample(n=num_of_samples(df_string_pairs))

Hamming
MLIPNS
Levenshtein
DamerauLevenshtein
Jaro
JaroWinkler
StrCmp95
NeedlemanWunsch
SmithWaterman
Jaccard
Sorensen
Tversky
Overlap
Tanimoto
Cosine
MongeElkan
Bag
LCSSeq
LCSStr
RatcliffObershelp
RLENCD
BWTRLENCD
SqrtNCD
EntropyNCD
BZ2NCD
LZMANCD
ZLIBNCD
MRA
Editex
Prefix
Postfix
Length
Identity
Matrix


,str1,str2,Hamming,MLIPNS,Levenshtein,DamerauLevenshtein,Jaro,JaroWinkler,StrCmp95,NeedlemanWunsch,SmithWaterman,Jaccard,Sorensen,Tversky,Overlap,Tanimoto,Cosine,MongeElkan,Bag,LCSSeq,LCSStr,RatcliffObershelp,RLENCD,BWTRLENCD,SqrtNCD,EntropyNCD,BZ2NCD,LZMANCD,ZLIBNCD,MRA,Editex,Prefix,Postfix,Length,Identity,Matrix
0,m700241001,,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,-inf,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.100000,0.000000,0.301030,0.000000,0.333333,0.375000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
4,m700241001,m006546749,0.300000,0.0,0.300000,0.400000,0.600000,0.600000,0.600000,0.650000,0.3,0.333333,0.500000,0.333333,0.5,-1.584963,0.500000,0.035,0.500000,0.400000,0.200000,0.400000,0.0,0.181818,0.341936,0.842349,0.800000,0.851852,0.375000,0.166667,0.300000,0.100000,0.000000,1.000000,0.0,0.0
3,m700241001,m006546756,0.300000,0.0,0.300000,0.400000,0.600000,0.600000,0.600000,0.650000,0.3,0.333333,0.500000,0.333333,0.5,-1.584963,0.500000,0.035,0.500000,0.400000,0.200000,0.400000,0.0,0.181818,0.360041,0.858762,0.800000,0.851852,0.375000,0.166667,0.300000,0.100000,0.000000,1.000000,0.0,0.0
7,m700241001,m006204687,0.300000,0.0,0.300000,0.400000,0.677778,0.677778,0.677778,0.650000,0.3,0.538462,0.700000,0.538462,0.7,-0.893085,0.700000,0.040,0.700000,0.500000,0.200000,0.500000,0.0,0.000000,0.420984,0.885525,0.800000,0.851852,0.375000,0.166667,0.400000,0.100000,0.000000,1.000000,0.0,0.0
9,m700241001,9790200207781,0.307692,0.0,0.384615,0.461538,0.698535,0.698535,0.698535,0.576923,0.4,0.437500,0.608696,0.437500,0.7,-1.192645,0.613941,0.040,0.538462,0.461538,0.230769,0.434783,0.0,0.076923,0.381748,0.909162,0.733333,0.793103,0.411765,0.333333,0.538462,0.000000,0.076923,0.769231,0.0,0.0
10,m700241001,m008060205,0.300000,0.0,0.400000,0.500000,0.677778,0.677778,0.677778,0.650000,0.3,0.428571,0.600000,0.428571,0.6,-1.222392,0.600000,0.030,0.600000,0.500000,0.200000,0.500000,0.0,-0.200000,0.324733,0.869203,0.766667,0.851852,0.375000,0.333333,0.400000,0.100000,0.000000,1.000000,0.0,0.0
5,m700241001,m200205343,0.400000,0.0,0.400000,0.400000,0.622222,0.622222,0.622222,0.700000,0.4,0.428571,0.600000,0.428571,0.6,-1.222392,0.600000,0.035,0.600000,0.500000,0.300000,0.500000,0.0,0.090909,0.388869,0.889834,0.833333,0.851852,0.500000,0.333333,0.400000,0.100000,0.000000,1.000000,0.0,0.0
1,m700241001,m006450510,0.200000,0.0,0.500000,0.600000,0.752381,0.777143,0.777143,0.700000,0.4,0.538462,0.700000,0.538462,0.7,-0.893085,0.700000,0.040,0.700000,0.600000,0.200000,0.600000,0.0,0.000000,0.408042,0.921053,0.800000,0.851852,0.375000,0.166667,0.500000,0.100000,0.000000,1.000000,0.0,0.0
11,m700241001,9790006201334,0.307692,0.0,0.384615,0.384615,0.698535,0.698535,0.698535,0.576923,0.3,0.533333,0.695652,0.533333,0.8,-0.906891,0.701646,0.045,0.615385,0.461538,0.153846,0.521739,0.0,0.071429,0.367413,0.863228,0.733333,0.793103,0.315789,0.333333,0.538462,0.000000,0.000000,0.769231,0.0,0.0
8,m700241001,m700241001,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.0,0.000000,1.000000,1.000,1.000000,1.000000,1.000000,1.000000,0.0,0.400000,0.585786,1.000000,0.966667,0.851852,0.875000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


### musicid

In [13]:
df_string_pairs = daf.string_pair_list(df_feature_base, 'musicid_x')

for algorithm in tedi_algorithms :
    if algorithm not in ['Gotoh', 'ArithNCD'] :
        daf.apply_similarities(df_string_pairs, tedi_algorithms[algorithm], algorithm)

df_string_pairs.sample(n=num_of_samples(df_string_pairs))

Hamming
MLIPNS
Levenshtein
DamerauLevenshtein
Jaro
JaroWinkler
StrCmp95
NeedlemanWunsch
SmithWaterman
Jaccard
Sorensen
Tversky
Overlap
Tanimoto
Cosine
MongeElkan
Bag
LCSSeq
LCSStr
RatcliffObershelp
RLENCD
BWTRLENCD
SqrtNCD
EntropyNCD
BZ2NCD
LZMANCD
ZLIBNCD
MRA
Editex
Prefix
Postfix
Length
Identity
Matrix


,str1,str2,Hamming,MLIPNS,Levenshtein,DamerauLevenshtein,Jaro,JaroWinkler,StrCmp95,NeedlemanWunsch,SmithWaterman,Jaccard,Sorensen,Tversky,Overlap,Tanimoto,Cosine,MongeElkan,Bag,LCSSeq,LCSStr,RatcliffObershelp,RLENCD,BWTRLENCD,SqrtNCD,EntropyNCD,BZ2NCD,LZMANCD,ZLIBNCD,MRA,Editex,Prefix,Postfix,Length,Identity,Matrix
46,245,134134,0.000000,0.0,0.166667,0.166667,0.500000,0.500000,0.500000,0.333333,0.000000,0.125000,0.222222,0.125000,0.333333,-3.000000,0.235702,0.055556,0.166667,0.166667,0.166667,0.222222,0.0,0.142857,0.160787,0.763170,0.833333,0.840000,0.666667,0.000000,0.166667,0.000000,0.000000,0.500000,0.0,0.0
18,245,1004,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.375000,0.000000,0.166667,0.285714,0.166667,0.333333,-2.584963,0.288675,0.055556,0.250000,0.250000,0.250000,0.285714,0.0,0.200000,0.171573,0.715305,0.875000,0.913043,0.600000,0.000000,0.250000,0.000000,0.000000,0.750000,0.0,0.0
47,245,453388,0.000000,0.0,0.166667,0.166667,0.666667,0.666667,0.666667,0.250000,0.000000,0.285714,0.444444,0.285714,0.666667,-1.807355,0.471405,0.111111,0.333333,0.333333,0.333333,0.444444,0.0,0.142857,0.242641,0.761479,0.846154,0.840000,0.500000,0.000000,0.500000,0.000000,0.000000,0.500000,0.0,0.0
13,245,4553,0.250000,0.0,0.250000,0.500000,0.722222,0.722222,0.722222,0.500000,0.000000,0.400000,0.571429,0.400000,0.666667,-1.321928,0.577350,0.111111,0.500000,0.500000,0.500000,0.571429,0.0,0.200000,0.371374,0.867553,0.869565,0.913043,0.600000,0.000000,0.500000,0.000000,0.000000,0.750000,0.0,0.0
41,245,8962,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.375000,0.000000,0.166667,0.285714,0.166667,0.333333,-2.584963,0.288675,0.055556,0.250000,0.250000,0.250000,0.285714,0.0,0.200000,0.146447,0.687774,0.920000,0.913043,0.600000,0.000000,0.000000,0.000000,0.000000,0.750000,0.0,0.0
66,245,00289,0.000000,0.0,0.200000,0.200000,0.000000,0.000000,0.000000,0.400000,0.333333,0.142857,0.250000,0.142857,0.333333,-2.807355,0.258199,0.055556,0.200000,0.200000,0.200000,0.250000,0.0,0.166667,0.132705,0.686838,0.884615,0.920000,0.545455,0.000000,0.400000,0.000000,0.000000,0.600000,0.0,0.0
27,245,601,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,-inf,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.250000,0.000000,0.613147,0.913043,0.913043,0.666667,0.000000,0.000000,0.000000,0.000000,1.000000,0.0,0.0
5,245,6646,0.000000,0.0,0.250000,0.250000,0.527778,0.527778,0.527778,0.500000,0.333333,0.166667,0.285714,0.166667,0.333333,-2.584963,0.288675,0.055556,0.250000,0.250000,0.250000,0.285714,0.0,0.200000,0.195262,0.601119,0.826087,0.913043,0.600000,0.333333,0.500000,0.000000,0.000000,0.750000,0.0,0.0
72,245,95088,0.000000,0.0,0.000000,0.200000,0.511111,0.511111,0.511111,0.300000,0.000000,0.142857,0.250000,0.142857,0.333333,-2.807355,0.258199,0.055556,0.200000,0.200000,0.200000,0.250000,0.0,0.166667,0.132705,0.686838,0.846154,0.920000,0.545455,0.000000,0.200000,0.000000,0.000000,0.600000,0.0,0.0
19,245,502430,0.000000,0.0,0.333333,0.333333,0.722222,0.722222,0.722222,0.416667,0.333333,0.500000,0.666667,0.500000,1.000000,-1.000000,0.707107,0.166667,0.500000,0.333333,0.333333,0.444444,0.0,0.142857,0.324583,0.785931,0.880000,0.840000,0.666667,0.000000,0.333333,0.000000,0.000000,0.500000,0.0,0.0


### part

In [14]:
df_string_pairs = daf.string_pair_list(df_feature_base, 'part_x')

for algorithm in tedi_algorithms :
    if algorithm not in ['Gotoh', 'ArithNCD'] :
        daf.apply_similarities(df_string_pairs, tedi_algorithms[algorithm], algorithm)

df_string_pairs.sample(n=num_of_samples(df_string_pairs))

Hamming
MLIPNS
Levenshtein
DamerauLevenshtein
Jaro
JaroWinkler
StrCmp95
NeedlemanWunsch
SmithWaterman
Jaccard
Sorensen
Tversky
Overlap
Tanimoto
Cosine
MongeElkan
Bag
LCSSeq
LCSStr
RatcliffObershelp
RLENCD
BWTRLENCD
SqrtNCD
EntropyNCD
BZ2NCD
LZMANCD
ZLIBNCD
MRA
Editex
Prefix
Postfix
Length
Identity
Matrix


,str1,str2,Hamming,MLIPNS,Levenshtein,DamerauLevenshtein,Jaro,JaroWinkler,StrCmp95,NeedlemanWunsch,SmithWaterman,Jaccard,Sorensen,Tversky,Overlap,Tanimoto,Cosine,MongeElkan,Bag,LCSSeq,LCSStr,RatcliffObershelp,RLENCD,BWTRLENCD,SqrtNCD,EntropyNCD,BZ2NCD,LZMANCD,ZLIBNCD,MRA,Editex,Prefix,Postfix,Length,Identity,Matrix
213,38 38,21 21,0.200000,0.0,0.200000,0.200000,0.466667,0.466667,0.466667,0.600000,0.2,0.111111,0.200000,0.111111,0.20,-3.169925,0.200000,0.02,0.200000,0.200000,0.200000,0.200000,0.0,0.166667,1.530097e-01,0.682782,0.846154,0.920000,0.545455,0.200000,0.200000,0.0,0.000,1.000000,0.0,0.0
10,38 38,71,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.0,0.000000,0.000000,0.000000,0.00,-inf,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.166667,1.110223e-16,0.509928,0.769231,0.920000,0.545455,0.000000,0.000000,0.0,0.000,0.400000,0.0,0.0
52,38 38,60 3 2015 02 01 432 437,0.086957,0.0,0.130435,0.130435,0.328986,0.328986,0.328986,0.173913,0.4,0.120000,0.214286,0.120000,0.60,-3.058894,0.279751,0.06,0.130435,0.130435,0.086957,0.142857,0.0,0.130435,1.247222e-01,0.616981,0.621622,0.696970,0.222222,0.166667,0.130435,0.0,0.000,0.217391,0.0,0.0
86,38 38,23 23 1932,0.100000,0.0,0.300000,0.300000,0.633333,0.633333,0.633333,0.400000,0.2,0.250000,0.400000,0.250000,0.60,-2.000000,0.424264,0.06,0.300000,0.300000,0.100000,0.400000,0.0,0.090909,2.315042e-01,0.733938,0.821429,0.851852,0.500000,0.166667,0.300000,0.0,0.000,0.500000,0.0,0.0
91,38 38,117,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.300000,0.0,0.000000,0.000000,0.000000,0.00,-inf,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.166667,1.110223e-16,0.471950,0.769231,0.920000,0.545455,0.000000,0.000000,0.0,0.000,0.600000,0.0,0.0
9,38 38,43,0.000000,0.0,0.200000,0.200000,0.566667,0.566667,0.566667,0.300000,0.0,0.166667,0.285714,0.166667,0.50,-2.584963,0.316228,0.04,0.200000,0.200000,0.200000,0.285714,0.0,0.166667,1.781836e-01,0.665981,0.769231,0.920000,0.545455,0.000000,0.200000,0.0,0.000,0.400000,0.0,0.0
15,38 38,12,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.0,0.000000,0.000000,0.000000,0.00,-inf,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.166667,1.110223e-16,0.509928,0.769231,0.920000,0.545455,0.000000,0.000000,0.0,0.000,0.400000,0.0,0.0
3,38 38,57 57,0.200000,0.0,0.200000,0.200000,0.466667,0.466667,0.466667,0.600000,0.2,0.111111,0.200000,0.111111,0.20,-3.169925,0.200000,0.02,0.200000,0.200000,0.200000,0.200000,0.0,0.166667,1.530097e-01,0.682782,0.846154,0.920000,0.545455,0.200000,0.200000,0.0,0.000,1.000000,0.0,0.0
197,38 38,14 14,0.200000,0.0,0.200000,0.200000,0.466667,0.466667,0.466667,0.600000,0.2,0.111111,0.200000,0.111111,0.20,-3.169925,0.200000,0.02,0.200000,0.200000,0.200000,0.200000,0.0,0.166667,1.530097e-01,0.682782,0.884615,0.920000,0.545455,0.200000,0.200000,0.0,0.000,1.000000,0.0,0.0
107,38 38,1 12 2 13 15,0.000000,0.0,0.166667,0.166667,0.522222,0.522222,0.522222,0.291667,0.2,0.133333,0.235294,0.133333,0.40,-2.906891,0.258199,0.06,0.166667,0.166667,0.083333,0.235294,0.0,0.000000,1.950436e-01,0.709224,0.827586,0.777778,0.333333,0.000000,0.166667,0.0,0.000,0.416667,0.0,0.0


### person

Attribute $\texttt{person}$ consists of three different representations of data, see chapter [Data Analysis](./1_DataAnalysis.ipynb). The similarity metric may be sensitive, depending on the kind of representation. All three representations will be investigated below.

In [15]:
person_representations = ['100', '245c', '700']

for pr in person_representations :
    df_string_pairs = daf.string_pair_list(df_feature_base, 'person_'+pr+'_x')

    print('\nperson_'+pr+'\n**********')
    for algorithm in tedi_algorithms :
        if algorithm not in ['Gotoh', 'ArithNCD'] :
            daf.apply_similarities(df_string_pairs, tedi_algorithms[algorithm], algorithm)

    display(df_string_pairs.sample(n=num_of_samples(df_string_pairs)))


person_100
**********
Hamming
MLIPNS
Levenshtein
DamerauLevenshtein
Jaro
JaroWinkler
StrCmp95
NeedlemanWunsch
SmithWaterman
Jaccard
Sorensen
Tversky
Overlap
Tanimoto
Cosine
MongeElkan
Bag
LCSSeq
LCSStr
RatcliffObershelp
RLENCD
BWTRLENCD
SqrtNCD
EntropyNCD
BZ2NCD
LZMANCD
ZLIBNCD
MRA
Editex
Prefix
Postfix
Length
Identity
Matrix


,str1,str2,Hamming,MLIPNS,Levenshtein,DamerauLevenshtein,Jaro,JaroWinkler,StrCmp95,NeedlemanWunsch,SmithWaterman,Jaccard,Sorensen,Tversky,Overlap,Tanimoto,Cosine,MongeElkan,Bag,LCSSeq,LCSStr,RatcliffObershelp,RLENCD,BWTRLENCD,SqrtNCD,EntropyNCD,BZ2NCD,LZMANCD,ZLIBNCD,MRA,Editex,Prefix,Postfix,Length,Identity,Matrix
29,bumandominique de,biglermatthias,0.058824,0.0,0.117647,0.117647,0.483754,0.483754,0.509804,0.470588,0.000000,0.240000,0.387097,0.240000,0.428571,-2.058894,0.388922,0.013841,0.352941,0.235294,0.117647,0.258065,0.0,0.055556,0.256628,0.877985,0.605263,0.741935,0.260870,0.166667,0.235294,0.058824,0.000000,0.823529,0.0,0.0
30,bumandominique de,schubertfranz,0.000000,0.0,0.000000,0.000000,0.430995,0.430995,0.471719,0.382353,0.000000,0.200000,0.333333,0.200000,0.384615,-2.321928,0.336336,0.013841,0.294118,0.176471,0.117647,0.200000,0.0,0.055556,0.238635,0.857075,0.578947,0.741935,0.260870,0.000000,0.147059,0.000000,0.000000,0.764706,0.0,0.0
15,bumandominique de,schlöndorffvolker,0.000000,0.0,0.117647,0.176471,0.490196,0.490196,0.560784,0.529412,0.117647,0.133333,0.235294,0.133333,0.235294,-2.906891,0.235294,0.012111,0.235294,0.235294,0.176471,0.235294,0.0,0.055556,0.186169,0.828121,0.487805,0.741935,0.291667,0.000000,0.235294,0.000000,0.000000,1.000000,0.0,0.0
34,bumandominique de,wagnerrichard,0.000000,0.0,0.176471,0.235294,0.430995,0.430995,0.485294,0.470588,0.076923,0.200000,0.333333,0.200000,0.384615,-2.321928,0.336336,0.015571,0.294118,0.235294,0.058824,0.200000,0.0,0.055556,0.252929,0.853535,0.578947,0.741935,0.260870,0.333333,0.294118,0.000000,0.000000,0.764706,0.0,0.0
31,bumandominique de,beethovenludwig van,0.052632,0.0,0.157895,0.210526,0.505547,0.505547,0.561275,0.500000,0.000000,0.440000,0.611111,0.440000,0.647059,-1.184425,0.612056,0.024221,0.578947,0.263158,0.105263,0.222222,0.0,0.050000,0.370053,0.892617,0.658537,0.741935,0.240000,0.166667,0.315789,0.052632,0.000000,0.894737,0.0,0.0
32,bumandominique de,wolfhugo,0.000000,0.0,0.117647,0.117647,0.289216,0.289216,0.362745,0.294118,0.000000,0.086957,0.160000,0.086957,0.250000,-3.523562,0.171499,0.005190,0.117647,0.117647,0.058824,0.160000,0.0,0.055556,0.101171,0.745876,0.552632,0.677419,0.260870,0.000000,0.176471,0.000000,0.000000,0.470588,0.0,0.0
12,bumandominique de,binderheidy,0.117647,0.0,0.294118,0.294118,0.587471,0.587471,0.617418,0.470588,0.272727,0.400000,0.571429,0.400000,0.727273,-1.321928,0.585018,0.015571,0.470588,0.294118,0.117647,0.357143,0.0,0.055556,0.278321,0.851165,0.631579,0.741935,0.260870,0.166667,0.411765,0.058824,0.000000,0.647059,0.0,0.0
38,bumandominique de,schlaurisimon,0.000000,0.0,0.117647,0.117647,0.507218,0.507218,0.520793,0.441176,0.000000,0.304348,0.466667,0.304348,0.538462,-1.716207,0.470871,0.017301,0.411765,0.235294,0.058824,0.200000,0.0,0.055556,0.300067,0.897599,0.631579,0.741935,0.260870,0.000000,0.235294,0.000000,0.000000,0.764706,0.0,0.0
22,bumandominique de,petrarcafrancesco,0.058824,0.0,0.058824,0.058824,0.450980,0.450980,0.521569,0.529412,0.058824,0.172414,0.294118,0.172414,0.294118,-2.536053,0.294118,0.010381,0.294118,0.176471,0.117647,0.176471,0.0,0.000000,0.209742,0.823696,0.526316,0.741935,0.260870,0.000000,0.235294,0.000000,0.000000,1.000000,0.0,0.0
36,bumandominique de,trieruri peter,0.000000,0.0,0.117647,0.176471,0.482726,0.482726,0.495752,0.470588,0.142857,0.240000,0.387097,0.240000,0.428571,-2.058894,0.388922,0.012111,0.352941,0.235294,0.058824,0.258065,0.0,0.111111,0.222952,0.777016,0.684211,0.741935,0.260870,0.000000,0.235294,0.000000,0.000000,0.823529,0.0,0.0



person_245c
**********
Hamming
MLIPNS
Levenshtein
DamerauLevenshtein
Jaro
JaroWinkler
StrCmp95
NeedlemanWunsch
SmithWaterman
Jaccard
Sorensen
Tversky
Overlap
Tanimoto
Cosine
MongeElkan
Bag
LCSSeq
LCSStr
RatcliffObershelp
RLENCD
BWTRLENCD
SqrtNCD
EntropyNCD
BZ2NCD
LZMANCD
ZLIBNCD
MRA
Editex
Prefix
Postfix
Length
Identity
Matrix


,str1,str2,Hamming,MLIPNS,Levenshtein,DamerauLevenshtein,Jaro,JaroWinkler,StrCmp95,NeedlemanWunsch,SmithWaterman,Jaccard,Sorensen,Tversky,Overlap,Tanimoto,Cosine,MongeElkan,Bag,LCSSeq,LCSStr,RatcliffObershelp,RLENCD,BWTRLENCD,SqrtNCD,EntropyNCD,BZ2NCD,LZMANCD,ZLIBNCD,MRA,Editex,Prefix,Postfix,Length,Identity,Matrix
245,"andreas kagermeier, tobias reeh (hrsg.)",jane austen ; mit einem nachw. von rudolf sühnel,0.041667,0.0,0.187500,0.250000,0.580128,0.580128,0.603365,0.500000,0.076923,0.403226,0.574713,0.403226,0.641026,-1.310340,0.577813,0.010519,0.520833,0.333333,0.041667,0.344828,0.0,0.020408,0.366331,0.925157,0.388889,0.446809,0.290909,0.166667,0.302083,0.0,0.000000,0.812500,0.0,0.0
242,"andreas kagermeier, tobias reeh (hrsg.)",hrsg. v. hans konrad biesalski ... [et al.] ; ...,0.012195,0.0,0.195122,0.207317,0.556139,0.556139,0.571274,0.335366,0.102564,0.344444,0.512397,0.344444,0.794872,-1.537657,0.548179,0.011834,0.378049,0.219512,0.060976,0.082645,0.0,-0.025974,0.372159,0.951933,0.329268,0.423729,0.305556,0.166667,0.280488,0.0,0.000000,0.475610,0.0,0.0
345,"andreas kagermeier, tobias reeh (hrsg.)",w.a. mozart ; klavierauszug zu 4 händen nach d...,0.010417,0.0,0.187500,0.187500,0.572115,0.572115,0.586538,0.296875,0.153846,0.336634,0.503704,0.336634,0.871795,-1.570749,0.555662,0.011506,0.354167,0.208333,0.031250,0.118519,0.0,0.031579,0.312083,0.919971,0.262626,0.333333,0.204545,0.000000,0.260417,0.0,0.000000,0.406250,0.0,0.0
117,"andreas kagermeier, tobias reeh (hrsg.)",rudolf steiner,0.025641,0.0,0.153846,0.153846,0.568043,0.568043,0.597164,0.256410,0.071429,0.261905,0.415094,0.261905,0.785714,-1.932886,0.470757,0.007890,0.282051,0.153846,0.051282,0.113208,0.0,0.025000,0.255691,0.888744,0.446429,0.512195,0.136364,0.000000,0.269231,0.0,0.000000,0.358974,0.0,0.0
383,"andreas kagermeier, tobias reeh (hrsg.)",by emanuel schikaneder ; musik von w.a. mozart...,0.010870,0.0,0.206522,0.217391,0.619669,0.619669,0.626971,0.315217,0.051282,0.364583,0.534351,0.364583,0.897436,-1.455679,0.584308,0.011834,0.380435,0.228261,0.032609,0.198473,0.0,0.032967,0.338795,0.928339,0.291667,0.343284,0.243902,0.000000,0.271739,0.0,0.000000,0.423913,0.0,0.0
393,"andreas kagermeier, tobias reeh (hrsg.)",[ed.: nicholas cronk... et al.]. 20 c,0.051282,0.0,0.051282,0.128205,0.474923,0.474923,0.501257,0.500000,0.027027,0.310345,0.473684,0.310345,0.486486,-1.688056,0.473848,0.010519,0.461538,0.282051,0.076923,0.210526,0.0,0.050000,0.392792,0.924654,0.508475,0.560976,0.272727,0.000000,0.192308,0.0,0.000000,0.948718,0.0,0.0
388,"andreas kagermeier, tobias reeh (hrsg.)",herausgeber: bert alexander petzold,0.051282,0.0,0.256410,0.282051,0.585836,0.585836,0.623785,0.576923,0.200000,0.480000,0.648649,0.480000,0.685714,-1.058894,0.649598,0.010191,0.615385,0.333333,0.076923,0.216216,0.0,0.025000,0.398001,0.945704,0.535714,0.512195,0.227273,0.000000,0.333333,0.0,0.000000,0.897436,0.0,0.0
142,"andreas kagermeier, tobias reeh (hrsg.)",von w. a. mozart ; clavier-auszug mit ital. un...,0.016129,0.0,0.209677,0.241935,0.556383,0.556383,0.585622,0.419355,0.102564,0.346667,0.514851,0.346667,0.666667,-1.528379,0.528744,0.011177,0.419355,0.258065,0.048387,0.198020,0.0,0.016129,0.347950,0.915256,0.351351,0.433962,0.265625,0.166667,0.306452,0.0,0.000000,0.629032,0.0,0.0
120,"andreas kagermeier, tobias reeh (hrsg.)",[regie:] volker schlöndorff,0.000000,0.0,0.230769,0.230769,0.506783,0.506783,0.550658,0.461538,0.222222,0.320000,0.484848,0.320000,0.592593,-1.643856,0.493067,0.009204,0.410256,0.282051,0.051282,0.090909,0.0,0.050000,0.331408,0.916306,0.431034,0.512195,0.136364,0.000000,0.307692,0.0,0.000000,0.692308,0.0,0.0
369,"andreas kagermeier, tobias reeh (hrsg.)",gustav mahler ; philadelphia orchestra ; dirig...,0.031746,0.0,0.206349,0.238095,0.573716,0.573716,0.594473,0.412698,0.076923,0.416667,0.588235,0.416667,0.769231,-1.263034,0.605228,0.010848,0.476190,0.238095,0.031746,0.235294,0.0,0.015625,0.375457,0.943226,0.369863,0.433962,0.274194,0.000000,0.277778,0.0,0.000000,0.6


person_700
**********
Hamming
MLIPNS
Levenshtein
DamerauLevenshtein
Jaro
JaroWinkler
StrCmp95
NeedlemanWunsch
SmithWaterman
Jaccard
Sorensen
Tversky
Overlap
Tanimoto
Cosine
MongeElkan
Bag
LCSSeq
LCSStr
RatcliffObershelp
RLENCD
BWTRLENCD
SqrtNCD
EntropyNCD
BZ2NCD
LZMANCD
ZLIBNCD
MRA
Editex
Prefix
Postfix
Length
Identity
Matrix


,str1,str2,Hamming,MLIPNS,Levenshtein,DamerauLevenshtein,Jaro,JaroWinkler,StrCmp95,NeedlemanWunsch,SmithWaterman,Jaccard,Sorensen,Tversky,Overlap,Tanimoto,Cosine,MongeElkan,Bag,LCSSeq,LCSStr,RatcliffObershelp,RLENCD,BWTRLENCD,SqrtNCD,EntropyNCD,BZ2NCD,LZMANCD,ZLIBNCD,MRA,Editex,Prefix,Postfix,Length,Identity,Matrix
139,"heilmannuwe, josumi, krausmichael, leitnerlott...",lapinibernardo,0.000000,0.0,0.090909,0.090909,0.488745,0.488745,0.496797,0.109091,0.071429,0.107143,0.193548,0.107143,0.857143,-3.222392,0.305788,0.002190,0.109091,0.090909,0.018182,0.096774,0.0,-0.009709,0.159653,0.805024,0.255102,0.295775,0.123596,0.000000,0.195455,0.000000,0.000000,0.127273,0.0,0.0
189,"heilmannuwe, josumi, krausmichael, leitnerlott...","schikanederemanuel, zallingermeinhard von",0.018182,0.0,0.200000,0.209091,0.608305,0.608305,0.615001,0.281818,0.097561,0.360360,0.529801,0.360360,0.975610,-1.472488,0.595623,0.004174,0.363636,0.227273,0.036364,0.264901,0.0,0.009709,0.389406,0.951969,0.397959,0.380282,0.258427,0.000000,0.322727,0.000000,0.000000,0.372727,0.0,0.0
23,"heilmannuwe, josumi, krausmichael, leitnerlott...","schlöndorffvolker, wurlitzerrudolph, frischmax...",0.036364,0.0,0.236364,0.236364,0.640642,0.640642,0.657326,0.495455,0.141176,0.523438,0.687179,0.523438,0.788235,-0.933911,0.692898,0.004463,0.609091,0.336364,0.036364,0.297436,0.0,0.019417,0.494098,0.954964,0.326531,0.436620,0.280899,0.166667,0.368182,0.000000,0.000000,0.772727,0.0,0.0
241,"heilmannuwe, josumi, krausmichael, leitnerlott...",zieglerleonhard,0.000000,0.0,0.109091,0.109091,0.559052,0.559052,0.574204,0.122727,0.066667,0.136364,0.240000,0.136364,1.000000,-2.874469,0.369274,0.002603,0.136364,0.109091,0.027273,0.144000,0.0,0.009709,0.206629,0.832623,0.295918,0.295775,0.146067,0.000000,0.213636,0.000000,0.000000,0.136364,0.0,0.0
6,"heilmannuwe, josumi, krausmichael, leitnerlott...",schikanederemanuel,0.018182,0.0,0.118182,0.118182,0.562408,0.562408,0.562408,0.140909,0.166667,0.163636,0.281250,0.163636,1.000000,-2.611435,0.404520,0.003099,0.163636,0.118182,0.027273,0.125000,0.0,0.009709,0.244377,0.868486,0.295918,0.323944,0.157303,0.000000,0.231818,0.000000,0.000000,0.163636,0.0,0.0
110,"heilmannuwe, josumi, krausmichael, leitnerlott...","mannionrosa, dessaynatalie, kitchenlinda, panz...",0.045455,0.0,0.236364,0.263636,0.656690,0.656690,0.684964,0.477273,0.088608,0.524194,0.687831,0.524194,0.822785,-0.931828,0.697274,0.004215,0.590909,0.327273,0.036364,0.222222,0.0,0.019417,0.466158,0.949974,0.357143,0.408451,0.325843,0.000000,0.395455,0.000000,0.000000,0.718182,0.0,0.0
162,"heilmannuwe, josumi, krausmichael, leitnerlott...","augustinusaurelius, cyrillus",0.036364,0.0,0.154545,0.163636,0.552886,0.552886,0.570808,0.204545,0.071429,0.232143,0.376812,0.232143,0.928571,-2.106915,0.468487,0.003430,0.236364,0.154545,0.027273,0.202899,0.0,-0.009709,0.274406,0.862163,0.316327,0.323944,0.202247,0.166667,0.277273,0.000000,0.000000,0.254545,0.0,0.0
227,"heilmannuwe, josumi, krausmichael, leitnerlott...","lauxgerd, deisterarno, schulte-körnegerd",0.000000,0.0,0.209091,0.218182,0.569156,0.569156,0.575974,0.286364,0.075000,0.327434,0.493333,0.327434,0.925000,-1.610726,0.557796,0.004132,0.336364,0.236364,0.045455,0.240000,0.0,0.019417,0.369397,0.961678,0.326531,0.352113,0.179775,0.000000,0.327273,0.000000,0.000000,0.363636,0.0,0.0
94,"heilmannuwe, josumi, krausmichael, leitnerlott...","schikanederemanuel, krusegeorg richard",0.018182,0.0,0.218182,0.227273,0.619325,0.619325,0.622865,0.281818,0.105263,0.345455,0.513514,0.345455,1.000000,-1.533432,0.587754,0.004050,0.345455,0.245455,0.045455,0.283784,0.0,-0.009709,0.371530,0.939429,0.357143,0.380282,0.269663,0.000000,0.345455,0.000000,0.000000,0.345455,0.0,0.0
175,"heilmannuwe, josumi, krausmichael, leitnerlott...",lustigjustus,0.000000,0.0,0.081818,0.081818,0.581313,0.581313,0.590556,0.095455,0.083333,0.109091,0.196721,0.109091,1.000000,-3.196397,0.330289,0.001529,0.109091,0.081818,0.027273,0.131148,0.0,0.000000,0.136160,0.695428,0.265306

### pubinit

In [16]:
df_string_pairs = daf.string_pair_list(df_feature_base, 'pubinit_x')

for algorithm in tedi_algorithms :
    if algorithm not in ['Gotoh', 'ArithNCD'] :
        daf.apply_similarities(df_string_pairs, tedi_algorithms[algorithm], algorithm)

df_string_pairs.sample(n=num_of_samples(df_string_pairs))

Hamming
MLIPNS
Levenshtein
DamerauLevenshtein
Jaro
JaroWinkler
StrCmp95
NeedlemanWunsch
SmithWaterman
Jaccard
Sorensen
Tversky
Overlap
Tanimoto
Cosine
MongeElkan
Bag
LCSSeq
LCSStr
RatcliffObershelp
RLENCD
BWTRLENCD
SqrtNCD
EntropyNCD
BZ2NCD
LZMANCD
ZLIBNCD
MRA
Editex
Prefix
Postfix
Length
Identity
Matrix


,str1,str2,Hamming,MLIPNS,Levenshtein,DamerauLevenshtein,Jaro,JaroWinkler,StrCmp95,NeedlemanWunsch,SmithWaterman,Jaccard,Sorensen,Tversky,Overlap,Tanimoto,Cosine,MongeElkan,Bag,LCSSeq,LCSStr,RatcliffObershelp,RLENCD,BWTRLENCD,SqrtNCD,EntropyNCD,BZ2NCD,LZMANCD,ZLIBNCD,MRA,Editex,Prefix,Postfix,Length,Identity,Matrix
44,könemann,e. eulenburg,0.166667,0.0,0.166667,0.166667,0.305556,0.305556,0.326389,0.416667,0.000000,0.111111,0.200000,0.111111,0.250000,-3.169925,0.204124,0.031250,0.166667,0.166667,0.083333,0.100000,0.0,0.153846,0.144300,0.747321,0.600000,0.777778,0.333333,0.000000,0.250000,0.000000,0.000000,0.666667,0.0,0.0
132,könemann,bartolomeo da cremona,0.000000,0.0,0.142857,0.142857,0.394841,0.394841,0.429365,0.261905,0.375000,0.160000,0.275862,0.160000,0.500000,-2.643856,0.308607,0.046875,0.190476,0.142857,0.095238,0.206897,0.0,0.045455,0.183797,0.736852,0.523810,0.696970,0.222222,0.000000,0.214286,0.000000,0.000000,0.380952,0.0,0.0
39,könemann,s. mode's verlag (gustav mode),0.000000,0.0,0.066667,0.066667,0.380556,0.380556,0.396389,0.166667,0.000000,0.085714,0.157895,0.085714,0.375000,-3.544321,0.193649,0.023438,0.100000,0.066667,0.033333,0.105263,0.0,0.032258,0.095510,0.652628,0.375000,0.621622,0.181818,0.166667,0.116667,0.000000,0.000000,0.266667,0.0,0.0
163,könemann,crw,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.187500,0.000000,0.000000,0.000000,0.000000,0.000000,-inf,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.111111,0.000000,0.576524,0.645161,0.851852,0.400000,0.000000,0.187500,0.000000,0.000000,0.375000,0.0,0.0
165,könemann,brunnen,0.125000,0.0,0.125000,0.250000,0.607143,0.607143,0.633929,0.500000,0.142857,0.363636,0.533333,0.363636,0.571429,-1.459432,0.534522,0.031250,0.500000,0.375000,0.250000,0.400000,0.0,0.222222,0.237728,0.820449,0.677419,0.851852,0.400000,0.200000,0.437500,0.000000,0.125000,0.875000,0.0,0.0
212,könemann,"in melanges, ed. van den heuvel. paris, gallimard",0.020408,0.0,0.102041,0.102041,0.568594,0.568594,0.568594,0.132653,0.250000,0.117647,0.210526,0.117647,0.750000,-3.087463,0.303046,0.046875,0.122449,0.102041,0.040816,0.140351,0.0,0.020000,0.121278,0.686905,0.344262,0.489362,0.113208,0.166667,0.142857,0.000000,0.000000,0.163265,0.0,0.0
190,könemann,penzance records,0.062500,0.0,0.125000,0.125000,0.500000,0.500000,0.518750,0.312500,0.000000,0.200000,0.333333,0.200000,0.500000,-2.321928,0.353553,0.039062,0.250000,0.187500,0.125000,0.250000,0.0,0.058824,0.171722,0.742999,0.513514,0.724138,0.272727,0.000000,0.156250,0.000000,0.000000,0.500000,0.0,0.0
146,könemann,sophia books,0.083333,0.0,0.083333,0.083333,0.402778,0.402778,0.423611,0.375000,0.000000,0.111111,0.200000,0.111111,0.250000,-3.169925,0.204124,0.015625,0.166667,0.083333,0.083333,0.100000,0.0,0.076923,0.115468,0.716364,0.628571,0.777778,0.333333,0.000000,0.208333,0.000000,0.000000,0.666667,0.0,0.0
147,könemann,bundesamt für landestopografie swisstopo,0.025000,0.0,0.125000,0.125000,0.516667,0.516667,0.516667,0.162500,0.000000,0.116279,0.208333,0.116279,0.625000,-3.104337,0.279508,0.046875,0.125000,0.125000,0.050000,0.208333,0.0,0.048780,0.111074,0.654391,0.360656,0.534884,0.121951,0.000000,0.175000,0.000000,0.000000,0.200000,0.0,0.0
16,könemann,universitätsverlag,0.000000,0.0,0.055556,0.111111,0.453704,0.453704,0.471759,0.250000,0.125000,0.130435,0.230769,0.130435,0.375000,-2.938599,0.250000,0.039062,0.166667,0.166667,0.055556,0.230769,0.0,0.052632,0.138071,0.701840,0.558140,0.741935,0.250000,0.000000,0.138889,0.000000,0.000000,0.444444,0.0,0.0


### scale

In [17]:
df_string_pairs = daf.string_pair_list(df_feature_base, 'scale_x')

for algorithm in tedi_algorithms :
    if algorithm not in ['Gotoh', 'ArithNCD'] :
        daf.apply_similarities(df_string_pairs, tedi_algorithms[algorithm], algorithm)

df_string_pairs.sample(n=num_of_samples(df_string_pairs))

Hamming
MLIPNS
Levenshtein
DamerauLevenshtein
Jaro
JaroWinkler
StrCmp95
NeedlemanWunsch
SmithWaterman
Jaccard
Sorensen
Tversky
Overlap
Tanimoto
Cosine
MongeElkan
Bag
LCSSeq
LCSStr
RatcliffObershelp
RLENCD
BWTRLENCD
SqrtNCD
EntropyNCD
BZ2NCD
LZMANCD
ZLIBNCD
MRA
Editex
Prefix
Postfix
Length
Identity
Matrix


,str1,str2,Hamming,MLIPNS,Levenshtein,DamerauLevenshtein,Jaro,JaroWinkler,StrCmp95,NeedlemanWunsch,SmithWaterman,Jaccard,Sorensen,Tversky,Overlap,Tanimoto,Cosine,MongeElkan,Bag,LCSSeq,LCSStr,RatcliffObershelp,RLENCD,BWTRLENCD,SqrtNCD,EntropyNCD,BZ2NCD,LZMANCD,ZLIBNCD,MRA,Editex,Prefix,Postfix,Length,Identity,Matrix
4,5000050000,5000050000,1.000000,1.0,1.00,1.00,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.00,1.000000,1.00,1.000000,1.000000,0.0,0.857143,0.585786,1.000000,0.961538,1.000000,1.00,1.000000,1.000000,1.000000,1.0,1.000000,1.0,1.0
2,5000050000,50000,0.500000,0.0,0.50,0.50,0.833333,0.900000,0.833333,0.500000,1.000000,0.500000,0.666667,0.500000,1.000000,-1.000000,0.707107,0.05,0.500000,0.50,0.500000,0.666667,0.0,0.571429,0.482362,1.000000,0.961538,0.925926,0.75,0.500000,0.900000,0.500000,0.5,0.500000,0.0,0.0
3,5000050000,50 000 8 10 8 35 45 55 46 05,0.142857,0.0,0.25,0.25,0.602381,0.602381,0.602381,0.303571,0.200000,0.266667,0.421053,0.266667,0.800000,-1.906891,0.478091,0.05,0.285714,0.25,0.107143,0.315789,0.0,-0.041667,0.187329,0.537651,0.513514,0.729730,0.30,0.333333,0.321429,0.071429,0.0,0.357143,0.0,0.0
0,5000050000,,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-inf,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.0,0.142857,0.000000,0.580744,0.000000,0.333333,0.50,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0
1,5000050000,100000,0.400000,0.0,0.50,0.50,0.777778,0.777778,0.777778,0.550000,0.666667,0.454545,0.625000,0.454545,0.833333,-1.137504,0.645497,0.04,0.500000,0.50,0.400000,0.625000,0.0,0.428571,0.343876,0.873183,0.846154,0.851852,0.75,0.250000,0.800000,0.000000,0.4,0.600000,0.0,0.0


### ttlfull

Attribute $\texttt{ttlfull}$ consists of two different representations of data as can be seen in chapter [Data Analysis](./1_DataAnalysis.ipynb). Both representations will be investigated below.

In [18]:
ttlfull_representations = ['245', '246']

for tf in ttlfull_representations :
    df_string_pairs = daf.string_pair_list(df_feature_base, 'ttlfull_'+tf+'_x')

    print('\nttlfull_'+tf+'\n***********')
    for algorithm in tedi_algorithms :
        if algorithm not in ['Gotoh', 'ArithNCD', 'Editex'] : # Some (very long string) samples fail with Editex
            daf.apply_similarities(df_string_pairs, tedi_algorithms[algorithm], algorithm)

    display(df_string_pairs.sample(n=num_of_samples(df_string_pairs)))


ttlfull_245
***********
Hamming
MLIPNS
Levenshtein
DamerauLevenshtein
Jaro
JaroWinkler
StrCmp95
NeedlemanWunsch
SmithWaterman
Jaccard
Sorensen
Tversky
Overlap
Tanimoto
Cosine
MongeElkan
Bag
LCSSeq
LCSStr
RatcliffObershelp
RLENCD
BWTRLENCD
SqrtNCD
EntropyNCD
BZ2NCD
LZMANCD
ZLIBNCD
MRA
Prefix
Postfix
Length
Identity
Matrix


,str1,str2,Hamming,MLIPNS,Levenshtein,DamerauLevenshtein,Jaro,JaroWinkler,StrCmp95,NeedlemanWunsch,SmithWaterman,Jaccard,Sorensen,Tversky,Overlap,Tanimoto,Cosine,MongeElkan,Bag,LCSSeq,LCSStr,RatcliffObershelp,RLENCD,BWTRLENCD,SqrtNCD,EntropyNCD,BZ2NCD,LZMANCD,ZLIBNCD,MRA,Prefix,Postfix,Length,Identity,Matrix
260,"bonne chance!, cours de langue française, deux...","bonne chance!, version realschule",0.125984,0.0,0.212598,0.212598,0.641808,0.641808,0.645626,0.236220,0.121212,0.250000,0.400000,0.250000,0.969697,-2.000000,0.494301,0.003255,0.251969,0.212598,0.118110,0.287500,0.0,0.033333,0.299286,0.926543,0.286957,0.382716,0.245283,0.500000,0.118110,0.007874,0.259843,0.0,0.0
51,"bonne chance!, cours de langue française, deux...",arts,0.000000,0.0,0.031496,0.031496,0.591207,0.591207,0.591207,0.031496,0.250000,0.031496,0.061069,0.031496,1.000000,-4.988685,0.177471,0.000899,0.031496,0.031496,0.007874,0.045802,0.0,0.016667,0.061827,0.580101,0.208696,0.259259,0.075472,0.000000,0.000000,0.000000,0.031496,0.0,0.0
115,"bonne chance!, cours de langue française, deux...","bonne chance!, cours de langue française, prem...",0.338583,0.0,0.480315,0.496063,0.780886,0.868531,0.868531,0.535433,0.186667,0.565891,0.722772,0.565891,0.973333,-0.821403,0.747981,0.003720,0.574803,0.503937,0.330709,0.613861,0.0,0.133333,0.462620,0.980475,0.504348,0.654321,0.556604,0.500000,0.330709,0.000000,0.590551,0.0,0.0
256,"bonne chance!, cours de langue française, deux...","die zauberflöte, eine deutsche oper in 2 aufzü...",0.023622,0.0,0.236220,0.244094,0.563654,0.563654,0.576363,0.342520,0.157895,0.343066,0.510870,0.343066,0.824561,-1.543443,0.552406,0.003627,0.370079,0.244094,0.031496,0.152174,0.0,0.033333,0.374259,0.964065,0.304348,0.308642,0.226415,0.000000,0.000000,0.000000,0.448819,0.0,0.0
283,"bonne chance!, cours de langue française, deux...","bonne chance!, cours de langue française, 3, l...",0.362205,0.0,0.448819,0.480315,0.764556,0.858733,0.867681,0.673228,0.382609,0.635135,0.776860,0.635135,0.817391,-0.654865,0.777816,0.003627,0.740157,0.543307,0.330709,0.504132,0.0,0.108333,0.487299,0.975819,0.382609,0.604938,0.452830,0.666667,0.330709,0.000000,0.905512,0.0,0.0
113,"bonne chance!, cours de langue française, deux...","neue ausgabe sämtlicher werke, die zauberflöte...",0.050000,0.0,0.240000,0.265000,0.662178,0.662178,0.672477,0.435000,0.125984,0.520930,0.685015,0.520930,0.881890,-0.940838,0.702751,0.003782,0.560000,0.305000,0.020000,0.250765,0.0,0.010471,0.396548,0.950124,0.160000,0.309735,0.182390,0.000000,0.000000,0.000000,0.635000,0.0,0.0
226,"bonne chance!, cours de langue française, deux...","die zauberflöte, oper in zwei aufzügen : kv.620",0.000000,0.0,0.173228,0.181102,0.488023,0.488023,0.511344,0.271654,0.063830,0.242857,0.390805,0.242857,0.723404,-2.041820,0.440076,0.003131,0.267717,0.188976,0.023622,0.206897,0.0,0.008333,0.307394,0.947828,0.234783,0.308642,0.160377,0.000000,0.000000,0.000000,0.370079,0.0,0.0
102,"bonne chance!, cours de langue française, deux...","die zauberflöte, grosse oper in zwei aufzügen",0.031496,0.0,0.188976,0.204724,0.551367,0.551367,0.566415,0.271654,0.133333,0.283582,0.441860,0.283582,0.844444,-1.818162,0.502661,0.003317,0.299213,0.196850,0.023622,0.232558,0.0,0.000000,0.327401,0.960132,0.295652,0.308642,0.207547,0.000000,0.000000,0.000000,0.354331,0.0,0.0
440,"bonne chance!, cours de langue française, deux...",katalog der graphischen porträts in der herzog...,0.022599,0.0,0.259887,0.271186,0.636817,0.636817,0.659808,0.485876,0.086614,0.512438,0.677632,0.512438,0.811024,-0.964551,0.686987,0.003689,0.581921,0.322034,0.022599,0.296053,0.0,0.029940,0.397935,0.933874,0.224490,0.402299,0.243902,0.166667,0.000000,0.000000,0.717514,0.0,0.0
248,"bonne chance!, cours de langue française, deux...",lebenswissenschaft praktische theologie?!,0.007874,0.0,0.157480,0.165354,0.521235,0.521235,0.534141,0.240157,0.097561,0.272727,0.428571,0.272727,0.878049,-1.874469,0.498894,0.003255,0.283465,0.173228,0.023622,0.142857,0.0,0.025000,0


ttlfull_246
***********
Hamming
MLIPNS
Levenshtein
DamerauLevenshtein
Jaro
JaroWinkler
StrCmp95
NeedlemanWunsch
SmithWaterman
Jaccard
Sorensen
Tversky
Overlap
Tanimoto
Cosine
MongeElkan
Bag
LCSSeq
LCSStr
RatcliffObershelp
RLENCD
BWTRLENCD
SqrtNCD
EntropyNCD
BZ2NCD
LZMANCD
ZLIBNCD
MRA
Prefix
Postfix
Length
Identity
Matrix


,str1,str2,Hamming,MLIPNS,Levenshtein,DamerauLevenshtein,Jaro,JaroWinkler,StrCmp95,NeedlemanWunsch,SmithWaterman,Jaccard,Sorensen,Tversky,Overlap,Tanimoto,Cosine,MongeElkan,Bag,LCSSeq,LCSStr,RatcliffObershelp,RLENCD,BWTRLENCD,SqrtNCD,EntropyNCD,BZ2NCD,LZMANCD,ZLIBNCD,MRA,Prefix,Postfix,Length,Identity,Matrix
43,medizinische informatik - kommunikation von ge...,"domodossola, arona",0.003704,0.0,0.059259,0.059259,0.559259,0.559259,0.559259,0.062963,0.000000,0.066667,0.125000,0.066667,1.000000,-3.906891,0.258199,0.000761,0.066667,0.059259,0.007407,0.097222,0.0,0.004049,0.106569,0.711208,0.140187,0.203252,0.074074,0.000000,0.000000,0.000000,0.066667,0.0,0.0
45,medizinische informatik - kommunikation von ge...,sinfonie nr. 1 d-dur,0.003704,0.0,0.070370,0.070370,0.533106,0.533106,0.533106,0.072222,0.000000,0.070111,0.131034,0.070111,0.950000,-3.834222,0.258557,0.001125,0.070370,0.070370,0.014815,0.117241,0.0,0.020243,0.130608,0.802938,0.144860,0.203252,0.079365,0.000000,0.000000,0.000000,0.074074,0.0,0.0
24,medizinische informatik - kommunikation von ge...,medizinische informatik - kommunikation von ge...,0.301205,0.0,0.725904,0.725904,0.806185,0.883711,0.885323,0.769578,0.662963,0.744928,0.853821,0.744928,0.951852,-0.424828,0.858385,0.001818,0.774096,0.728916,0.271084,0.800664,0.0,0.402062,0.525521,0.989049,0.666667,0.748148,0.691943,0.833333,0.271084,0.003012,0.813253,0.0,0.0
68,medizinische informatik - kommunikation von ge...,informatique de santé - communication entre di...,0.029644,0.0,0.369565,0.373518,0.679188,0.679188,0.680892,0.445652,0.437037,0.512671,0.677835,0.512671,0.974074,-0.963896,0.711539,0.001831,0.519763,0.409091,0.177866,0.347938,0.0,0.280303,0.469564,0.994095,0.440789,0.604790,0.529412,0.333333,0.000000,0.001976,0.533597,0.0,0.0
63,medizinische informatik - kommunikation von ge...,informatique de santé - communication entre di...,0.042644,0.0,0.366738,0.368870,0.678930,0.678930,0.683016,0.458422,0.466667,0.533195,0.695535,0.533195,0.951852,-0.907265,0.722212,0.001831,0.547974,0.424307,0.191898,0.519621,0.0,0.340659,0.484506,0.993553,0.533808,0.651613,0.576000,0.333333,0.000000,0.002132,0.575693,0.0,0.0
51,medizinische informatik - kommunikation von ge...,journal of adam mickiewicz university,0.003704,0.0,0.092593,0.092593,0.524752,0.524752,0.533971,0.114815,0.027027,0.124542,0.221498,0.124542,0.918919,-3.005294,0.340170,0.001317,0.125926,0.092593,0.011111,0.071661,0.0,0.000000,0.199541,0.904552,0.126168,0.235772,0.111111,0.000000,0.000000,0.000000,0.137037,0.0,0.0
0,medizinische informatik - kommunikation von ge...,,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-inf,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.004049,0.000000,0.176358,0.000000,0.073171,0.031746,0.000000,0.000000,0.000000,0.000000,0.0,0.0
55,medizinische informatik - kommunikation von ge...,medizinische informatik - kommunikation von ge...,0.243176,0.0,0.607940,0.607940,0.762334,0.857401,0.861112,0.638958,0.414815,0.610048,0.757801,0.610048,0.944444,-0.713006,0.773047,0.001831,0.632754,0.620347,0.230769,0.725111,0.0,0.336134,0.509063,0.988548,0.529412,0.643312,0.597561,1.000000,0.223325,0.014888,0.669975,0.0,0.0
61,medizinische informatik - kommunikation von ge...,informatique de santé - communication entre di...,0.036260,0.0,0.320611,0.326336,0.667151,0.667151,0.669396,0.409351,0.325926,0.489681,0.657431,0.489681,0.966667,-1.030086,0.693894,0.001831,0.498092,0.375954,0.171756,0.471033,0.0,0.308458,0.468194,0.992965,0.480392,0.597633,0.517986,0.333333,0.000000,0.001908,0.515267,0.0,0.0
9,medizinische informatik - kommunikation von ge...,"die zauberflöte, ausgabe für gesang und klavier",0.014815,0.0,0.122222,0.122222,0.564191,0.564191,0.566689,0.148148,0.042553,0.174074,0.296530,0.174074,1.000000,-2.522227,0.417222,0.001221,0.174074,0.122222,0.018519,0.151420,0.0,0.008097,0.225545,0.884403,0.177570,0.252033,0.153439,0.166667,0.000000,0.000000,0.174074,0.0,0.0


### volumes

In [19]:
df_string_pairs = daf.string_pair_list(df_feature_base, 'volumes_x')

for algorithm in tedi_algorithms :
    if algorithm not in ['Gotoh', 'ArithNCD'] :
        daf.apply_similarities(df_string_pairs, tedi_algorithms[algorithm], algorithm)

df_string_pairs.sample(n=num_of_samples(df_string_pairs))

Hamming
MLIPNS
Levenshtein
DamerauLevenshtein
Jaro
JaroWinkler
StrCmp95
NeedlemanWunsch
SmithWaterman
Jaccard
Sorensen
Tversky
Overlap
Tanimoto
Cosine
MongeElkan
Bag
LCSSeq
LCSStr
RatcliffObershelp
RLENCD
BWTRLENCD
SqrtNCD
EntropyNCD
BZ2NCD
LZMANCD
ZLIBNCD
MRA
Editex
Prefix
Postfix
Length
Identity
Matrix


,str1,str2,Hamming,MLIPNS,Levenshtein,DamerauLevenshtein,Jaro,JaroWinkler,StrCmp95,NeedlemanWunsch,SmithWaterman,Jaccard,Sorensen,Tversky,Overlap,Tanimoto,Cosine,MongeElkan,Bag,LCSSeq,LCSStr,RatcliffObershelp,RLENCD,BWTRLENCD,SqrtNCD,EntropyNCD,BZ2NCD,LZMANCD,ZLIBNCD,MRA,Editex,Prefix,Postfix,Length,Identity,Matrix
23,301,1 221,0.000000,0.0,0.200000,0.200000,0.000000,0.000000,0.000000,0.400000,0.333333,0.142857,0.250000,0.142857,0.333333,-2.807355,0.258199,0.055556,0.200000,0.200000,0.200000,0.250000,0.00,0.166667,0.178184,0.754847,0.807692,0.920000,0.545455,0.000000,0.400000,0.000000,0.200000,0.600000,0.0,0.0
260,301,1 423,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.300000,0.000000,0.333333,0.500000,0.333333,0.666667,-1.584963,0.516398,0.111111,0.400000,0.200000,0.200000,0.250000,0.00,0.166667,0.234315,0.724546,0.814815,0.920000,0.545455,0.000000,0.000000,0.000000,0.000000,0.600000,0.0,0.0
185,301,1 47,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.375000,0.000000,0.166667,0.285714,0.166667,0.333333,-2.584963,0.288675,0.055556,0.250000,0.250000,0.250000,0.285714,0.00,0.200000,0.146447,0.687774,0.851852,0.913043,0.600000,0.000000,0.000000,0.000000,0.000000,0.750000,0.0,0.0
149,301,105,0.333333,1.0,0.333333,0.333333,0.555556,0.555556,0.555556,0.666667,0.333333,0.500000,0.666667,0.500000,0.666667,-1.000000,0.666667,0.111111,0.666667,0.333333,0.333333,0.333333,0.00,0.250000,0.390524,0.871049,0.913043,0.913043,0.666667,0.333333,0.333333,0.000000,0.000000,1.000000,0.0,0.0
228,301,1 113,0.200000,0.0,0.200000,0.400000,0.511111,0.511111,0.511111,0.400000,0.000000,0.333333,0.500000,0.333333,0.666667,-1.584963,0.516398,0.111111,0.400000,0.200000,0.200000,0.250000,0.00,0.166667,0.353113,0.853364,0.807692,0.920000,0.545455,0.250000,0.400000,0.000000,0.000000,0.600000,0.0,0.0
145,301,211,0.333333,1.0,0.333333,0.666667,0.555556,0.555556,0.555556,0.666667,0.333333,0.200000,0.333333,0.200000,0.333333,-2.321928,0.333333,0.055556,0.333333,0.333333,0.333333,0.333333,0.00,0.250000,0.227388,0.661819,0.826087,0.913043,0.666667,0.000000,0.333333,0.000000,0.333333,1.000000,0.0,0.0
155,301,347,0.333333,1.0,0.333333,0.333333,0.555556,0.555556,0.555556,0.666667,0.333333,0.200000,0.333333,0.200000,0.333333,-2.321928,0.333333,0.055556,0.333333,0.333333,0.333333,0.333333,0.00,0.250000,0.195262,0.742098,0.913043,0.913043,0.666667,0.333333,0.333333,0.333333,0.000000,1.000000,0.0,0.0
255,301,1 346,0.000000,0.0,0.200000,0.200000,0.000000,0.000000,0.000000,0.400000,0.333333,0.333333,0.500000,0.333333,0.666667,-1.584963,0.516398,0.111111,0.400000,0.200000,0.200000,0.250000,0.00,0.166667,0.234315,0.724546,0.814815,0.920000,0.545455,0.000000,0.200000,0.000000,0.000000,0.600000,0.0,0.0
37,301,379,0.333333,1.0,0.333333,0.333333,0.555556,0.555556,0.555556,0.666667,0.333333,0.200000,0.333333,0.200000,0.333333,-2.321928,0.333333,0.055556,0.333333,0.333333,0.333333,0.333333,0.00,0.250000,0.195262,0.742098,0.913043,0.913043,0.666667,0.333333,0.333333,0.333333,0.000000,1.000000,0.0,0.0
32,301,1 412,0.000000,0.0,0.200000,0.200000,0.511111,0.511111,0.511111,0.400000,0.000000,0.142857,0.250000,0.142857,0.333333,-2.807355,0.258199,0.055556,0.200000,0.200000,0.200000,0.250000,0.00,0.166667,0.154538,0.719132,0.814815,0.920000,0.545455,0.000000,0.200000,0.000000,0.000000,0.600000,0.0,0.0
